<a href="https://colab.research.google.com/github/anksont/my-work/blob/main/Project_role_of_dairy_in_livelihood_improvement_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing necessary dependencies

In [1]:
!pip install colab-env --upgrade
!pip install --upgrade plotly
!pip install -U kaleido
!pip install plotly>=4.7.1
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

  Created wheel for colab-env: filename=colab_env-0.2.0-py3-none-any.whl size=3836 sha256=3c52734eaf2d27728f27d6a6cab97753644911036c78f19fcb67a33f6e23e3a9
  Stored in directory: /root/.cache/pip/wheels/bb/ca/e8/3d25b6abb4ac719ecb9e837bb75f2a9b980430005fb12a9107
Successfully built colab-env
     |████████████████████████████████| 25.3 MB 1.4 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 79.9 MB 93 kB/s 
--2021-12-09 04:39:00--  https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/99037241/9dc3a580-286a-11e9-8a21-4312b7c8a512?X-Amz-Algorithm=AWS4-HMAC-SHA25

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import datetime
import re
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
from google.colab import files

#Mounting of Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Reading xlsx file 

In [4]:
#import psycopg2 as ps

In [5]:
#df=pd.read_csv('/content/drive/MyDrive/data_file/Data entry_18_Dec_2020_updated.xlsx',mangle_dupe_cols=True,header=1)
# last file path used for cleaning
#df1=pd.read_excel('/content/drive/MyDrive/data_file/Datafile_24_Apr_2021.xlsx',sheet_name="Sheet1",mangle_dupe_cols=True,header=0)
#for excel
df=pd.read_excel('/content/drive/MyDrive/data_file/cleaned Data file (Avi-Sup)/cleaned file1.xlsx',sheet_name="Sheet1",mangle_dupe_cols=True,header=0)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
print(df.shape)


(1926, 399)


In [8]:
# Reseting index from 1 to len(df)
df.reset_index(drop=True,inplace=True)
df.shape

(1926, 399)

In [9]:
df = df.loc[:, df.columns.notnull()]

In [10]:
df.index # dimension of data

RangeIndex(start=0, stop=1926, step=1)

In [11]:
def checking_types_in_df(df=df,colname=[]):
  for i in colname:
    d=[]
    if df[i].dtypes=='object':
      for m in df[i]:
        d.append(type(m))
    print(f'{i}-{pd.Series(d).value_counts()}')

In [12]:
df.columns

Index(['Unnamed: 0', 'Cooperative 1 (Member=1 /Non-Member =0)',
       '1_ Name of Head of Household (HH)', 'Zones', 'State', 'Milk Union',
       'Taluka', 'DCS* Name', 'District', 'Village',
       ...
       '80 _ You have attended training programme organized by dairy society and it has benefitted to you.',
       '81 _ Other women also participate in the various training programmes conducted by the society.',
       '82 _ Now you have women representatives in the village Panchayat which actively participate in the decision making process',
       '83 _ Procuring Agency* has women representatives on its board.',
       '84 _ You maintain dairy accounts in a book and it has proved to be useful.',
       '85 _ Your dairy society helps you in raising bank loan for dairy purposes.',
       '86 _ Your bank lend you money for the purchase of livestock regularly, as the loan installment is regularly deducted from your sale proceeds by the society and paid directly to the financing bank.',

In [13]:
df.columns[0:20]

Index(['Unnamed: 0', 'Cooperative 1 (Member=1 /Non-Member =0)',
       '1_ Name of Head of Household (HH)', 'Zones', 'State', 'Milk Union',
       'Taluka', 'DCS* Name', 'District', 'Village', '2_Gender ',
       '3_Age (years)', '4_Education (Code)', '5_ Social Group (Code)',
       '6_Income Group (BPL/APL)', '8_House Structure (Code)',
       '9_Occupation - (a) Principal (Code)', '(b) Subsidiary (Code)',
       '10_Experience in Dairying (Years)', '11_Formal Training in Dairying '],
      dtype='object')

# Socio-economics of HH



In [14]:
df['Cooperative 1 (Member=1 /Non-Member =0)'].value_counts()
#code 2.0 substitution 

1    1014
0     912
Name: Cooperative 1 (Member=1 /Non-Member =0), dtype: int64

In [15]:
df['Total_land_ownership'][df['Total_land_ownership'].isna()]=0
df['(b) Subsidiary (Code)'][df['(b) Subsidiary (Code)'].isna()]=0

In [16]:
lko=df[['Cooperative 1 (Member=1 /Non-Member =0)','3_Age (years)','8_House Structure (Code)','6_Income Group (BPL/APL)','2_Gender ', 'State_dummy','Cooperative 1 (Member=1 /Non-Member =0)','9_Occupation - (a) Principal (Code)','(b) Subsidiary (Code)','Total_land_ownership','10_Experience in Dairying (Years)','Total cattle livestock at HH']]
import numpy as np
from sklearn.impute import KNNImputer
X =lko.apply(lambda x:pd.to_numeric(x,errors='coerce'))
imputer = KNNImputer(n_neighbors=10)
df[lko.columns]=pd.DataFrame(imputer.fit_transform(X),columns=X.columns)
df[['3_Age (years)','6_Income Group (BPL/APL)','2_Gender ', 'State_dummy','8_House Structure (Code)','Cooperative 1 (Member=1 /Non-Member =0)','9_Occupation - (a) Principal (Code)','(b) Subsidiary (Code)']]=np.round(df[['3_Age (years)','6_Income Group (BPL/APL)','2_Gender ', 'State_dummy','8_House Structure (Code)','Cooperative 1 (Member=1 /Non-Member =0)','9_Occupation - (a) Principal (Code)','(b) Subsidiary (Code)']])

In [17]:
df['Total_land_ownership'][df['Total_land_ownership']==0]=np.nan
df['(b) Subsidiary (Code)'][df['(b) Subsidiary (Code)']==0]=np.nan

In [18]:
dist=pd.DataFrame(df.groupby(['Zones','State','Cooperative 1 (Member=1 /Non-Member =0)'])['Cooperative 1 (Member=1 /Non-Member =0)'].count())
dist.columns=['Sampling Distribution']
dist.reset_index(inplace=True,drop=False)
dist.loc[dist['Cooperative 1 (Member=1 /Non-Member =0)']==0,'Cooperative 1 (Member=1 /Non-Member =0)']='Non-Member'
dist.loc[dist['Cooperative 1 (Member=1 /Non-Member =0)']==1,'Cooperative 1 (Member=1 /Non-Member =0)']='Member'
import plotly.express as px
fig = px.sunburst(dist, path=['Zones','State','Cooperative 1 (Member=1 /Non-Member =0)','Sampling Distribution'], values='Sampling Distribution',color_discrete_sequence=['Green','Brown','orange','Purple','blue'],color_continuous_midpoint='Sampling Distribution',)
fig.update_layout(title={'text':'State-wise Data Sample collection','x':0.5,'y':0.99})
fig.update_layout(height=880,width=1000)
fig.show()
fig.write_image('State-wise Data Sample collection.pdf')
files.download('State-wise Data Sample collection.pdf')
dist.to_excel('State-wise Data Sample collection.xlsx')
files.download('State-wise Data Sample collection.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
dist=pd.DataFrame(df.groupby(['Zones','State','Milk Union','Cooperative 1 (Member=1 /Non-Member =0)'])['Cooperative 1 (Member=1 /Non-Member =0)'].count())
dist.columns=['Sampling Distribution']
dist.reset_index(inplace=True,drop=False)
dist.loc[dist['Cooperative 1 (Member=1 /Non-Member =0)']==0,'Cooperative 1 (Member=1 /Non-Member =0)']='Non-Member'
dist.loc[dist['Cooperative 1 (Member=1 /Non-Member =0)']==1,'Cooperative 1 (Member=1 /Non-Member =0)']='Member'
import plotly.express as px
fig = px.sunburst(dist, path=['Zones','Milk Union','Cooperative 1 (Member=1 /Non-Member =0)','Sampling Distribution'], values='Sampling Distribution',color_discrete_sequence=['Green','Brown','orange','Purple','blue'],color_continuous_midpoint='Sampling Distribution',)
fig.update_layout(title={'text':'Union-wise Data Sample Collection','x':0.5,'y':0.99})
fig.update_layout(height=1000,width=1200)
fig.show()
fig.write_image('Union-wise Data Sample Collection.pdf')
files.download('Union-wise Data Sample Collection.pdf')
dist.to_excel('Union-wise Data Sample Collection.xlsx')
files.download('Union-wise Data Sample Collection.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
dist=pd.DataFrame(df.groupby(['2_Gender ','Cooperative 1 (Member=1 /Non-Member =0)'])['Cooperative 1 (Member=1 /Non-Member =0)'].count())
dist.columns=['Sampling Distribution']
dist.reset_index(inplace=True,drop=False)
dist.loc[dist['Cooperative 1 (Member=1 /Non-Member =0)']==0,'Cooperative 1 (Member=1 /Non-Member =0)']='Non-Member'
dist.loc[dist['Cooperative 1 (Member=1 /Non-Member =0)']==1,'Cooperative 1 (Member=1 /Non-Member =0)']='Member'
dist.loc[dist['2_Gender ']==2,'2_Gender ']='Female'
dist.loc[dist['2_Gender ']==1,'2_Gender ']='Male'
import plotly.express as px
fig = px.sunburst(dist, path=['2_Gender ','Cooperative 1 (Member=1 /Non-Member =0)','Sampling Distribution'], values='Sampling Distribution',color_discrete_sequence=['Green','orange','Purple','lightblue'],color_continuous_midpoint='Sampling Distribution',)
fig.update_layout(title={'text':'Genderwise Data Sample Collection','x':0.5,'y':0.95})
fig.update_layout(height=600,width=600)
fig.show()
fig.write_image('Genderwise Data Sample Collection.pdf')
files.download('Genderwise Data Sample Collection.pdf')
dist.to_excel('Genderwise Data Sample Collection.xlsx')
files.download('Genderwise Data Sample Collection.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
training=pd.DataFrame(df.groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)','11_Formal Training in Dairying '])['11_Formal Training in Dairying '].count())
training.columns=['Train/untrained Count']
training['Train/untrained Percentage']=np.nan
training.reset_index(drop=False,inplace=True)
for i in training['Zones'].unique():
  for j in training['Cooperative 1 (Member=1 /Non-Member =0)'].unique():
    training.loc[(training['Zones']==i) & (training['Cooperative 1 (Member=1 /Non-Member =0)']==j),'Train/untrained Percentage']=training.loc[(training['Zones']==i) & (training['Cooperative 1 (Member=1 /Non-Member =0)']==j),'Train/untrained Count']/training[(training['Zones']==i) & (training['Cooperative 1 (Member=1 /Non-Member =0)']==j)].sum()[3]
training['Train/untrained Percentage']=np.round(training['Train/untrained Percentage']*100,2)
m=training[training.sort_values(['Cooperative 1 (Member=1 /Non-Member =0)','11_Formal Training in Dairying '])['11_Formal Training in Dairying ']==1]
try:
  m.drop('11_Formal Training in Dairying ',inplace=True,axis=1)
except:
  print('Error')

In [22]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(
    x=m['Zones'][m['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=m['Train/untrained Percentage'][m['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=m['Train/untrained Percentage'][m['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    name='Non-Member',
    marker_color='lightblue'
))
fig.add_trace(go.Bar(
    x=m['Zones'][m['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=m['Train/untrained Percentage'][m['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=m['Train/untrained Percentage'][m['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    name='Member',
    marker_color='coral'
))
fig.update_traces(texttemplate='%{text:.2s}%', textposition='outside')
fig.update_layout(title={'text':'<b>Dairy Related Training percentage of Member and Non-Member in Zone</b>','x':0.50,'y':0.95})
fig.update_layout(barmode='group', xaxis_tickangle=0)
fig.update_yaxes(title=dict(text='Dairy related training (in %)'))
fig.update_layout(template='simple_white')
fig.show()
fig.write_image('Dairy Related Training percentage of Member and Non-Member in Zone.pdf')
files.download('Dairy Related Training percentage of Member and Non-Member in Zone.pdf')
m.to_excel('Dairy Related Training percentage of Member and Non-Member in Zone.xlsx')
files.download('Dairy Related Training percentage of Member and Non-Member in Zone.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
edu_dict=defaultdict(lambda:'College')
edu_list=['illiterate','Primary','Middle ','Secondary','Sr. Secondary','College']
for i,j in enumerate(sorted(df['4_Education (Code)'].unique())[:-1]):
  edu_dict[j]=edu_list[i]
df['4_Education (Code)']=df['4_Education (Code)'].map(edu_dict)

In [24]:
m=pd.DataFrame(df.groupby(['Cooperative 1 (Member=1 /Non-Member =0)','4_Education (Code)'])['4_Education (Code)'].count()).rename(columns={'4_Education (Code)':'count'}).reset_index(drop=False)
pd.pivot(m,index='4_Education (Code)',columns='Cooperative 1 (Member=1 /Non-Member =0)',values='count')

Cooperative 1 (Member=1 /Non-Member =0),0.0,1.0
4_Education (Code),,
College,75,99
Middle,166,180
Primary,215,216
Secondary,205,251
Sr. Secondary,107,115
illiterate,144,153


In [25]:
h_dict=defaultdict(lambda:'Semi-Pucca')
h_list=['Pucca','Semi-Pucca','Kuchha']
for i,j in enumerate(sorted(df['8_House Structure (Code)'].unique())[0:3]):
  h_dict[j]=h_list[i]
df['8_House Structure (Code)']=df['8_House Structure (Code)'].map(h_dict)

In [26]:
print(dict(df['8_House Structure (Code)'].value_counts(dropna=False)))
mw=pd.DataFrame(df.groupby(['Cooperative 1 (Member=1 /Non-Member =0)','8_House Structure (Code)'])['8_House Structure (Code)'].count()).rename(columns={'8_House Structure (Code)':'count'}).reset_index(drop=False)
pd.pivot(mw,index='8_House Structure (Code)',columns='Cooperative 1 (Member=1 /Non-Member =0)',values='count')

{'Pucca': 1211, 'Semi-Pucca': 563, 'Kuchha': 152}


Cooperative 1 (Member=1 /Non-Member =0),0.0,1.0
8_House Structure (Code),,
Kuchha,74,78
Pucca,540,671
Semi-Pucca,298,265


In [27]:
p_dict=defaultdict(lambda:'APL')
p_list=['BPL','APL']
for i,j in enumerate(sorted(df['6_Income Group (BPL/APL)'].unique())[0:2]):
  p_dict[j]=p_list[i]
df['6_Income Group (BPL/APL)']=df['6_Income Group (BPL/APL)'].map(p_dict)

In [28]:
print(dict(df['6_Income Group (BPL/APL)'].value_counts(dropna=False)))
pl=pd.DataFrame(df.groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)','6_Income Group (BPL/APL)'])['6_Income Group (BPL/APL)'].count()).rename(columns={'6_Income Group (BPL/APL)':'pl_count'}).reset_index(drop=False)
pl['pl %']=np.nan
for i in pl['Zones'].unique():
  for j in pl['Cooperative 1 (Member=1 /Non-Member =0)'].unique():
    pl.loc[(pl['Zones']==i) & (pl['Cooperative 1 (Member=1 /Non-Member =0)']==j),'pl %']=pl.loc[(training['Zones']==i) & (pl['Cooperative 1 (Member=1 /Non-Member =0)']==j),'pl_count']/pl[(pl['Zones']==i) & (pl['Cooperative 1 (Member=1 /Non-Member =0)']==j)].sum()[3]
pl['pl %']=np.round(pl['pl %']*100,2)
m=pl[pl.sort_values(['Cooperative 1 (Member=1 /Non-Member =0)','6_Income Group (BPL/APL)'])['6_Income Group (BPL/APL)']=='APL']


{'APL': 1265, 'BPL': 661}


In [29]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    x=m['Zones'][m['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=m['pl %'][m['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=m['pl %'][m['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    name='Non-Member',
    marker_color='lightblue'
))
fig.add_trace(go.Bar(
    x=m['Zones'][m['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=m['pl %'][m['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=m['pl %'][m['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    name='Member',
    marker_color='coral'
))
fig.update_traces(texttemplate='%{text:.3s}%', textposition='outside')
fig.update_layout(title={'text':'Zonewise APL Member and Non-Member Percentage','x':0.50,'y':0.95})
fig.update_layout(barmode='group', xaxis_tickangle=0)
fig.update_yaxes(title=dict(text='Above Poverty line (in %)'))
fig.update_layout(template='simple_white')
fig.show()
fig.write_image('Zonewise APL Member and Non-Member Percentage.pdf')
files.download('Zonewise APL Member and Non-Member Percentage.pdf')
m.to_excel('Zonewise APL Member and Non-Member Percentage.xlsx')
files.download('Zonewise APL Member and Non-Member Percentage.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
e_dict=defaultdict(lambda:'Others')
e_list=['Cultivator','Dairy & AH','Agri-labour','Non Farm Labor','Own Non-Farm Est.','Trade','Employee in Service','Others']
for i,j in enumerate(sorted(df['9_Occupation - (a) Principal (Code)'].unique())[0:8]):
  e_dict[j]=e_list[i]
df['9_Occupation - (a) Principal (Code)']=df['9_Occupation - (a) Principal (Code)'].map(e_dict)

In [31]:
m=pd.DataFrame(df.groupby(['Cooperative 1 (Member=1 /Non-Member =0)','9_Occupation - (a) Principal (Code)'])['9_Occupation - (a) Principal (Code)'].count()).rename(columns={'9_Occupation - (a) Principal (Code)':'Prin_Occ_count'}).reset_index(drop=False)
m.pivot(index='9_Occupation - (a) Principal (Code)',columns='Cooperative 1 (Member=1 /Non-Member =0)',values='Prin_Occ_count')

Cooperative 1 (Member=1 /Non-Member =0),0.0,1.0
9_Occupation - (a) Principal (Code),,
Agri-labour,42,40
Cultivator,595,653
Dairy & AH,223,282
Employee in Service,11,8
Non Farm Labor,20,5
Others,6,18
Own Non-Farm Est.,5,4
Trade,10,4


In [32]:
sorted(df['(b) Subsidiary (Code)'].unique())[1:9]

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]

In [33]:
e_dict=defaultdict(lambda:np.nan)
e_list=['Cultivator','Dairy & AH','Agri-labour','Non Farm Labor','Own Non-Farm Est.','Trade','Employee in Service','Others']
for i,j in enumerate(sorted(df['(b) Subsidiary (Code)'].unique())[1:9]):
  e_dict[j]=e_list[i]
df['(b) Subsidiary (Code)']=df['(b) Subsidiary (Code)'].map(e_dict)

In [34]:
m=pd.DataFrame(df.groupby(['Cooperative 1 (Member=1 /Non-Member =0)','(b) Subsidiary (Code)'])['(b) Subsidiary (Code)'].count()).rename(columns={'(b) Subsidiary (Code)':'Sec_Occ_count'}).reset_index(drop=False)
m.pivot(index='(b) Subsidiary (Code)',columns='Cooperative 1 (Member=1 /Non-Member =0)',values='Sec_Occ_count')

Cooperative 1 (Member=1 /Non-Member =0),0.0,1.0
(b) Subsidiary (Code),,
Agri-labour,43,56
Cultivator,53,68
Dairy & AH,501,520
Employee in Service,3,2
Non Farm Labor,10,9
Others,2,1
Own Non-Farm Est.,4,3
Trade,3,5


In [35]:
df.loc[:,'3_Age (years)':'11_Formal Training in Dairying '].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1926 entries, 0 to 1925
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   3_Age (years)                        1926 non-null   float64
 1   4_Education (Code)                   1926 non-null   object 
 2   5_ Social Group (Code)               1926 non-null   int64  
 3   6_Income Group (BPL/APL)             1926 non-null   object 
 4   8_House Structure (Code)             1926 non-null   object 
 5   9_Occupation - (a) Principal (Code)  1926 non-null   object 
 6   (b) Subsidiary (Code)                1283 non-null   object 
 7   10_Experience in Dairying (Years)    1926 non-null   float64
 8   11_Formal Training in Dairying       1926 non-null   int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 135.5+ KB


In [36]:
#df[["State",'Cooperative 1 (Member=1 /Non-Member =0)','8_House Structure (Code)']][(df["8_House Structure (Code)"]>3.0)|(df['8_House Structure (Code)'].isna())].fillna(-1).groupby(['State','Cooperative 1 (Member=1 /Non-Member =0)','8_House Structure (Code)'])['8_House Structure (Code)'].count()

In [37]:
try:
  df.drop('7_Mobile Number',inplace=True,axis=1)
except:
  print("column is dropped")

column is dropped


In [38]:
for i in df.iloc[:,df.columns.get_loc('4_Education (Code)'):df.columns.get_loc('11_Formal Training in Dairying ')+1].columns:
  if len(df[i].unique()) <16:
    print(f'{i :}'+f'{dict(df[i].value_counts(dropna=False))}')

4_Education (Code){'Secondary': 456, 'Primary': 431, 'Middle ': 346, 'illiterate': 297, 'Sr. Secondary': 222, 'College': 174}
5_ Social Group (Code){3: 811, 4: 777, 2: 207, 1: 131}
6_Income Group (BPL/APL){'APL': 1265, 'BPL': 661}
8_House Structure (Code){'Pucca': 1211, 'Semi-Pucca': 563, 'Kuchha': 152}
9_Occupation - (a) Principal (Code){'Cultivator': 1248, 'Dairy & AH': 505, 'Agri-labour': 82, 'Non Farm Labor': 25, 'Others': 24, 'Employee in Service': 19, 'Trade': 14, 'Own Non-Farm Est.': 9}
(b) Subsidiary (Code){'Dairy & AH': 1021, nan: 643, 'Cultivator': 121, 'Agri-labour': 99, 'Non Farm Labor': 19, 'Trade': 8, 'Own Non-Farm Est.': 7, 'Employee in Service': 5, 'Others': 3}
11_Formal Training in Dairying {2: 1157, 1: 769}


In [39]:
try:
  df.drop('Family Size',axis=1,inplace=True)
except:
  print('Columns in dropped')

In [40]:
df.iloc[:,10:30].columns

Index(['2_Gender ', '3_Age (years)', '4_Education (Code)',
       '5_ Social Group (Code)', '6_Income Group (BPL/APL)',
       '8_House Structure (Code)', '9_Occupation - (a) Principal (Code)',
       '(b) Subsidiary (Code)', '10_Experience in Dairying (Years)',
       '11_Formal Training in Dairying ', '< 18 Years_Male',
       '< 18 Years_Female', '18-60 Years_Male', '18-60 Years_Female',
       '> 60 Years _Male', '> 60 Years _Female', 'Male', 'Female',
       'Informal farmers’ group', 'Cooperative'],
      dtype='object')

#<h1>Working on family size</h1>

In [41]:
df['Zones'].unique()

array(['Western Zone', 'Northern Zone', 'Southern Zone', 'Eastern Zone',
       'North Eastern Zone'], dtype=object)

In [42]:
try:
  df.insert(df.columns.get_loc('Female')+1,"Family Size",df[['Male','Female']].sum(axis=1))
except: 
  print("column is inserted")
#df[["Male","Female","Family Size"]][df["Family Size"]>= 10]

In [43]:
df["Male"]=df[["< 18 Years_Male","18-60 Years_Male",'> 60 Years _Male']].apply(lambda x: x.sum(),axis=1)
df["Female"]=df[["< 18 Years_Female","18-60 Years_Female",'> 60 Years _Female']].apply(lambda x: x.sum(),axis=1)
df["Family Size"]=df["Male"]+df["Female"]

In [44]:
fl=["Zones","Cooperative 1 (Member=1 /Non-Member =0)",'Male','Female',"Family Size"]
flk=pd.melt(df[fl],id_vars=["Zones","Cooperative 1 (Member=1 /Non-Member =0)"],value_vars=['Male','Female','Family Size'])

In [45]:
flk['Zones'].unique()

array(['Western Zone', 'Northern Zone', 'Southern Zone', 'Eastern Zone',
       'North Eastern Zone'], dtype=object)

In [46]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd


fig = make_subplots(rows=1, cols=1,subplot_titles=['Family Size Distribution'])

fig.add_trace(go.Violin(x=flk["Zones"][(flk['variable'] == 'Family Size')& (flk["Cooperative 1 (Member=1 /Non-Member =0)"]==0)],
                        y=flk['value'][(flk['variable'] == 'Family Size') & (flk["Cooperative 1 (Member=1 /Non-Member =0)"]==0)],
                        legendgroup='FZ', scalegroup='FZ', name=f'Family Size (Non-Mem)',side='positive',),row=1,col=1
             )
fig.add_trace(go.Violin(x=flk["Zones"][(flk['variable'] == 'Family Size')& (flk["Cooperative 1 (Member=1 /Non-Member =0)"]==1)],
                        y=flk['value'][(flk['variable'] == 'Family Size') & (flk["Cooperative 1 (Member=1 /Non-Member =0)"]==1)],
                        legendgroup='FZ', scalegroup='FZ', name=f'Family Size (Mem)',side='negative'),row=1,col=1
             )
fig.update_layout(violinmode='overlay',height=600)
fig.update_traces(box_visible=False, meanline_visible=True,)
fig.update_layout(template='simple_white')
fig.show()
fig.write_image('Family Size Distribution.pdf')
files.download('Family Size Distribution.pdf')
flk.to_excel('Family Size Distribution.xlsx')
files.download('Family Size Distribution.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
fam_dyn_list=["State","Zones",'Cooperative 1 (Member=1 /Non-Member =0)',"< 18 Years_Male","18-60 Years_Male",'> 60 Years _Male',"Male","< 18 Years_Female","18-60 Years_Female",'> 60 Years _Female',"Female","Family Size"]

In [48]:
para_cal={}
for i in fam_dyn_list[3:]:
  para_cal[i]=["count","mean","std","min","max"]
#para_cal

In [49]:
#np.round(df[fam_dyn_list].groupby(["State","Cooperative 1 (Member=1 /Non-Member =0)"]).agg(para_cal),2).to_excel("1-Statewise Family Dynamics of member and non-mem.xlsx")
np.round(df[fam_dyn_list].groupby(["Zones","Cooperative 1 (Member=1 /Non-Member =0)"]).agg(para_cal),2).to_excel("1-Zonewise Family Dynamics of member and non-mem.xlsx")
#files.download("1-Statewise Family Dynamics of member and non-mem.xlsx")
#files.download("1-Zonewise Family Dynamics of member and non-mem.xlsx")
np.round(df[fam_dyn_list].groupby(["Zones","Cooperative 1 (Member=1 /Non-Member =0)"]).agg(para_cal),2)

< 18 Years_Male  ... Family Size
                                                                     count  ...         max
Zones              Cooperative 1 (Member=1 /Non-Member =0)                  ...            
Eastern Zone       0.0                                                 114  ...          17
                   1.0                                                 117  ...          19
North Eastern Zone 0.0                                                  76  ...           8
                   1.0                                                  79  ...           8
Northern Zone      0.0                                                 241  ...          15
                   1.0                                                 248  ...          17
Southern Zone      0.0                                                 228  ...          12
                   1.0                                                 231  ...           7
Western Zone       0.0                                                 253  ...          14
                   1.0                                                 339  ...          12

[10 rows x 45 columns]

#<h1>Working on Membership of the Organization(s)</h1>
Variable list=['Informal farmers’ group', 'Cooperative', 'Shelf Help Group (SHG)','Farmer Producer Company', 'Any other Farmer Producer Organisation',
       'NGO - National level', 'NGO - International level',
       'Village Panchayat', 'Zilla Parishad/Block level/Panchyat samiti',
       'MLA/MP'

In [50]:
df.iloc[:,df.columns.get_loc('Informal farmers’ group'):df.columns.get_loc('MLA/MP')+1]=df.iloc[:,df.columns.get_loc('Informal farmers’ group'):df.columns.get_loc('MLA/MP')+1].apply(lambda x:pd.to_numeric(x,errors="coerce"),axis=1)
df.iloc[:,df.columns.get_loc('Informal farmers’ group'):df.columns.get_loc('MLA/MP')+1]=df.iloc[:,df.columns.get_loc('Informal farmers’ group'):df.columns.get_loc('MLA/MP')+1].fillna(0)

In [51]:
#df.iloc[:,df.columns.get_loc('Informal farmers’ group'):df.columns.get_loc('MLA/MP')+1].head()

In [52]:
for i in df.iloc[:,df.columns.get_loc('Informal farmers’ group'):df.columns.get_loc('MLA/MP')+1].columns:
  print(f"{i}-{dict(df[i].value_counts(dropna=False))}")

Informal farmers’ group-{0: 1864, 1: 62}
Cooperative-{1: 1436, 0: 489, 2: 1}
Shelf Help Group (SHG)-{0: 1653, 1: 273}
Farmer Producer Company-{0: 1900, 1: 26}
Any other Farmer Producer Organisation-{0: 1883, 1: 43}
NGO - National level-{0: 1926}
NGO - International level-{0: 1926}
Village Panchayat-{0: 1643, 1: 283}
Zilla Parishad/Block level/Panchyat samiti-{0: 1925, 1: 1}
MLA/MP-{0: 1926}


#<h1>Working on operational land holdings</h1>
<p>Variable_list=['Irrigated_Code', 'Irrigated_Owned', 'Irrigated_Leased in',
       'Irrigated_Leased out', 'Irrigated_Total',
       'Irrigated_Source of irrigation (Code)', 'Unirrigated_Code',
       'Unirrigated_Owned', 'Unirrigated_Leased in', 'Unirrigated_Leased out',
       'Unirrigated_Total', 'Unirrigated_Source of irrigation (Code)']</p>

In [53]:
#for i in df.loc[:,'Irrigated_Code':'Total_land_ownership'].columns:
  #print(f"{i}-{dict(df[i].value_counts())}")

In [54]:
df.rename(columns={'Total_land_ownership':'Total Cultivate land owned or Leased by HH'},inplace=True)

In [55]:
al=["Zones",'Cooperative 1 (Member=1 /Non-Member =0)','Irrigated_Total','Unirrigated_Total','Total Cultivate land owned or Leased by HH']
al_dict=dict()
for i in al[2:]:
  al_dict[i]=['count','mean','std','min','max']
np.round(df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']).agg(al_dict),2)

Irrigated_Total  ... Total Cultivate land owned or Leased by HH
                                                                     count  ...                                        max
Zones              Cooperative 1 (Member=1 /Non-Member =0)                  ...                                           
Eastern Zone       0.0                                                  87  ...                                      32.11
                   1.0                                                  92  ...                                      34.58
North Eastern Zone 0.0                                                  44  ...                                      26.00
                   1.0                                                  60  ...                                      39.00
Northern Zone      0.0                                                 223  ...                                      76.00
                   1.0                                                 231  ...                                      82.00
Southern Zone      0.0                                                 130  ...                                      60.00
                   1.0                                                 148  ...                                      20.00
Western Zone       0.0                                                 191  ...                                      32.11
                   1.0                                                 264  ...                                      45.00

[10 rows x 15 columns]

In [56]:
gh=df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)'])[['Irrigated_Total','Unirrigated_Total','Total Cultivate land owned or Leased by HH']].mean()
gh.reset_index(inplace=True)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.graph_objects as go
cols=gh.columns[2:]
fig = make_subplots(rows=len(cols), cols=1, subplot_titles=['Average Irrigated Land in HH','Average Unirrigated Land in HH','Average total land owned or leased by HH'])
for i,j in enumerate(cols):
  fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=np.round(gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],2),
    name='Non-Member',showlegend=True if i==len(cols)-1 else False,
    marker_color='lightsalmon'
  ),row=i+1,col=1)
  fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=np.round(gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],2),
    name='Member',showlegend=True if i==len(cols)-1 else False,
    marker_color='darkblue'
  ),row=i+1,col=1)
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0,template='simple_white',title={'text':'<b>Zonewise land owned or leased by HH (in Acre)</b>','x':0.45,'y':0.95})
fig.show()
fig.write_image('Zonewise land owned or leased by HH (in Acre).pdf')
files.download('Zonewise land owned or leased by HH (in Acre).pdf')
flk.to_excel('Family Size Distribution.xlsx')
files.download('Family Size Distribution.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#<h1>Working on the Dairy Related livestock</h1>
In Milk_Count (No)
<p>(columns index range (144:174))</p>

In [57]:
l_l=["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']+list(df.iloc[:,df.columns.get_loc('In Milk_Count (No)'):df.columns.get_loc('In Milk_Count (No)')+35].columns)
cl=["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']+l_l[2:12]

In [58]:
def live_owned_cost(df=df,l_l=l_l,d='Cow',pos=[2,10]):
  cl=["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']+l_l[pos[0]:pos[1]]
  k_dict=dict()
  m_dict=dict()
  k=[i for i in cl if 'Count (No)' in i]
  m=[i for i in cl if 'Animal (Rs)'in i]
  for i,j in zip(k,m):
    k_dict[i]=['count','mean','std','min','max']
    m_dict[j]=['count','mean','std','min','max']
  np.round(df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']).agg(k_dict),2).to_excel(f'1-cost of {d} in HH.xlsx')
  np.round(df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']).agg(k_dict),2).to_excel(f'1-{d} owned by in HH.xlsx')
  files.download(f'1-cost of {d} in HH.xlsx')
  files.download(f'1-{d} owned by in HH.xlsx')

In [59]:
#cow
##live_owned_cost()
#buffalo
##live_owned_cost(df=df,l_l=l_l,d='local cow',pos=[12,22])
#local_cow
##live_owned_cost(df=df,l_l=l_l,d='Buffalo',pos=[22,32])

In [60]:
k_dict={}
for i in l_l[-5:]:
  k_dict[i]=['count','mean','std','min','max']
df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']).agg(k_dict)
df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']).agg(k_dict).to_excel('Complete live stock Dynamics in HH.xlsx')
df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']).agg(k_dict)
files.download('Complete live stock Dynamics in HH.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [61]:
import scipy.stats as stats

In [62]:
#df['Total cattle livestock at HH'][((df['Zones'] == i) & (df['Cooperative 1 (Member=1 /Non-Member =0)']==1))]
df['Total cattle livestock at HH'][((df['Zones'] == i) & (df['Cooperative 1 (Member=1 /Non-Member =0)']==0))]

Series([], Name: Total cattle livestock at HH, dtype: float64)

In [63]:
t_test_for_ls_in_hh=dict()

In [64]:
def t_test_calculator(col_name=['Zones','Cooperative 1 (Member=1 /Non-Member =0)'],eva_col='Total cattle livestock at HH',output_file='T-test for Average Animal holding in HH.xlsx'):
  for i in df['Zones'].unique():
    t_test_for_ls_in_hh[i]=list(stats.ttest_ind(df[eva_col][(df[col_name[0]] == i) & (df[col_name[1]]==1) & (df[eva_col]>0)],
                          df[eva_col][(df[col_name[0]] == i) & (df[col_name[1]]==0) & (df[eva_col]>0)])[0:2])
  np.round(pd.DataFrame(t_test_for_ls_in_hh).T.rename(columns={0:"t-test",1:'p-value'}),3).to_excel(output_file)
  files.download(output_file)

In [65]:
t_test_calculator()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [66]:
for i in df['Zones'].unique():
  print(i,"-",stats.ttest_ind(df['Total In Milk animal at HH'][(df['Zones'] == i) & (df['Cooperative 1 (Member=1 /Non-Member =0)']==1) & (df['Total In Milk animal at HH']>0)],
                        df['Total In Milk animal at HH'][(df['Zones'] == i) & (df['Cooperative 1 (Member=1 /Non-Member =0)']==0) & (df['Total In Milk animal at HH']>0)]))

Western Zone - Ttest_indResult(statistic=1.211191310131073, pvalue=0.22630864692628447)
Northern Zone - Ttest_indResult(statistic=2.213526155041748, pvalue=0.027324085992703206)
Southern Zone - Ttest_indResult(statistic=3.3794658474728902, pvalue=0.0007903706817160167)
Eastern Zone - Ttest_indResult(statistic=0.5342281894717807, pvalue=0.59371823828135)
North Eastern Zone - Ttest_indResult(statistic=3.308064258859327, pvalue=0.001171121071452929)


In [67]:
df[l_l[-5:]].columns
cols=['Total In Milk animal at HH','Total cattle livestock at HH']

In [68]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd


fig = make_subplots(rows=len(cols), cols=1,subplot_titles=['In Milk Animal in HH','Total Animal in HH'],)
for j,i in enumerate(cols):
  fig.add_trace(go.Violin(x=df["Zones"][(df["Cooperative 1 (Member=1 /Non-Member =0)"]==0)],
                        y=df[i][(df["Cooperative 1 (Member=1 /Non-Member =0)"]==0)],
                        legendgroup=f'FZ-{i}', scalegroup='FZ', name=f'{i} (Non-Mem)',side='negative',),row=j+1,col=1
             )
  fig.add_trace(go.Violin(x=df["Zones"][ (df["Cooperative 1 (Member=1 /Non-Member =0)"]==1)],
                        y=df[i][(df["Cooperative 1 (Member=1 /Non-Member =0)"]==1)],
                        legendgroup=f'FZ-{i}', scalegroup='FZ', name=f'{i} (Mem)',side='positive'),row=j+1,col=1
             )
#fig.update_traces(yaxis=dict(tickmode='linear',tick0=0,dtick=10))
fig.update_layout(violinmode='overlay', violingap=0,violingroupgap=0,height=1000,width=1200)
fig.update_traces(box_visible=False, meanline_visible=True,)

fig.update_layout(template='simple_white')
fig.show()

<h1> Working on Other Livestock Inventory (Total count)</h1>

In [69]:
olive_l=[df.columns.get_loc('State'), df.columns.get_loc('District'), df.columns.get_loc("Taluka"),df.columns.get_loc("Cooperative 1 (Member=1 /Non-Member =0)")]
olive_l+=list(np.arange(df.columns.get_loc('Goat'),df.columns.get_loc('Total Count (No.)')+1,1))
#print(df.iloc[:,olive_l].replace(0,np.nan).groupby(f).describe(include=np.number).dropna(thresh=len(olive_l)))
df['Total Count (No.)']=df['Total Count (No.)'].astype('float')
df['Total Count (No.)']=df.iloc[:,olive_l[4:-2]].apply(lambda x: np.sum(x),axis=1)
df['Total Count (No.)']=df['Total Count (No.)'].replace(0,np.nan).astype('float')
#print(df.iloc[:,olive_l[0:len(olive_l)-2]].columns)
df.iloc[:,olive_l[2:]].describe()
#df.iloc[:,olive_l][df['Poultry']>=75]

,Cooperative 1 (Member=1 /Non-Member =0),Goat,Sheep,Poultry,Pig,Total Count (No.)
count,1926.000000,290.000000,38.000000,177.000000,11.000000,401.000000
mean,0.526480,4.617241,9.763158,138.079096,13.000000,65.568579
std,0.499428,7.464915,15.137986,955.087933,29.738863,637.095170
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,2.000000,2.000000,5.000000,1.000000,2.000000
50%,1.000000,2.000000,4.000000,10.000000,1.000000,5.000000
75%,1.000000,5.000000,10.000000,24.000000,5.000000,14.000000
max,1.000000,100.000000,80.000000,12000.000000,100.000000,12003.000000


In [70]:
#df.iloc[:,olive_l].info()

In [71]:
df['Production_per_livestock']=np.round(df['Production_per_livestock'],1)

#<h2>Production and Disposal of Milk and Milk Product<h2>

Local Cow

In [72]:
ml=[i for i in df.columns if 'In Milk_Count (No)' in i]
mp=[i for i in df.columns if 'milk drawn in pail (liters)' in i][0:3]
mq=[i for i in df.columns if 'Milk Sale (liters)' in i][0:3]
df[ml+mp+mq]=df[ml+mp+mq].replace(0,np.nan)

In [73]:
#df.loc[:,'Local Cow_Kept at Home (liters)':]

In [74]:
k_list=[ml,mp,mq,['Cow','Cross-Breed','Buffalo']]
for i in range(0,3):
  h_dict={}
  for k in [k_list[0][i],k_list[1][i],k_list[2][i]]:
    h_dict[k]=['count','mean','std','min','max']
  np.round(df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']).agg(h_dict),2).to_excel(f'2- In Milk {k_list[3][i]} and Milk Produce in HH.xlsx')
  files.download(f'2- In Milk {k_list[3][i]} and Milk Produce in HH.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [75]:
k_list=[ml,mp,mq,['Cow','Cross-Breed','Buffalo']]
for i in range(0,3):
  h_dict={}
  print([k_list[0][i],k_list[1][i],k_list[2][i]])


['In Milk_Count (No)', 'Local Cow_Quantity of milk drawn in pail (liters)', 'Local Cow_Milk Sale (liters)']
['In Milk_Count (No).1', 'Cross bred Cow_Quantity of milk drawn in pail (liters)', 'Cross bred Cow_Milk Sale (liters)']
['In Milk_Count (No).2', 'Buffalo_Quantity of milk drawn in pail (liters)', 'Buffalo_Milk Sale (liters)']


In [76]:
gh=df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)'])[['In Milk_Count (No)', 'Local Cow_Quantity of milk drawn in pail (liters)', 'Local Cow_Milk Sale (liters)']].mean()
gh.reset_index(inplace=True)
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
cols=gh.columns[2:]
fig = make_subplots(rows=len(cols), cols=1, subplot_titles=['Avg. In Milk local Cow', 'Avg. Quantity of milk drawn in pail (liters)', 'Avg. Milk for Sale (liters)'])
for i,j in enumerate(cols):
  fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=np.round(gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],2),
    name='Non-Member',showlegend=True if i==len(cols)-1 else False,
    marker_color='lightsalmon'
  ),row=i+1,col=1)
  fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=np.round(gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],2),
    name='Member',showlegend=True if i==len(cols)-1 else False,
    marker_color='darkblue'
  ),row=i+1,col=1)
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0,template='simple_white',title={'text':'<b>Local Cow Milk Production Dynamics in HH</b>','x':0.50,'y':0.95})
fig.show()
fig.write_image('Local Cow Milk Production Dynamics in HH.pdf')
files.download('Local Cow Milk Production Dynamics in HH.pdf')
gh.to_excel('Local Cow Milk Production Dynamics in HH.xlsx')
files.download('Local Cow Milk Production Dynamics in HH.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [77]:
gh=df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)'])[['In Milk_Count (No).1', 'Cross bred Cow_Quantity of milk drawn in pail (liters)', 'Cross bred Cow_Milk Sale (liters)']].mean()
gh.reset_index(inplace=True)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.graph_objects as go
cols=gh.columns[2:]
fig = make_subplots(rows=len(cols), cols=1, subplot_titles=['Avg. In Milk Cross Breed', 'Avg. Quantity of milk drawn in pail (liters)', 'Avg. Milk for Sale (liters)'])
for i,j in enumerate(cols):
  fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=np.round(gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],2),
    name='Non-Member',showlegend=True if i==len(cols)-1 else False,
    marker_color='lightsalmon'
  ),row=i+1,col=1)
  fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=np.round(gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],2),
    name='Member',showlegend=True if i==len(cols)-1 else False,
    marker_color='darkblue'
  ),row=i+1,col=1)
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0,template='simple_white',title={'text':'<b>Cross Breed Milk Production Dynamics in HH</b>','x':0.50,'y':0.95})
fig.show()
fig.write_image('Cross Breed Milk Production Dynamics in HH.pdf')
files.download('Cross Breed Milk Production Dynamics in HH.pdf')
gh.to_excel('Cross Breed Milk Production Dynamics in HH.xlsx')
files.download('Cross Breed Milk Production Dynamics in HH.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [78]:
gh=df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)'])[['In Milk_Count (No).2', 'Buffalo_Quantity of milk drawn in pail (liters)', 'Buffalo_Milk Sale (liters)']].mean()
gh.reset_index(inplace=True)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.graph_objects as go
cols=gh.columns[2:]
fig = make_subplots(rows=len(cols), cols=1, subplot_titles=['Avg. In Milk Buffalo', 'Avg. Quantity of milk drawn in pail (liters)', 'Avg. Milk for Sale (liters)'])
for i,j in enumerate(cols):
  fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=np.round(gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],2),
    name='Non-Member',showlegend=True if i==len(cols)-1 else False,
    marker_color='lightsalmon'
  ),row=i+1,col=1)
  fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=np.round(gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],2),
    name='Member',showlegend=True if i==len(cols)-1 else False,
    marker_color='darkblue'
  ),row=i+1,col=1)
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0,template='simple_white',title={'text':'<b>Buffalo Milk Production Dynamics in HH</b>','x':0.45,'y':0.95})
fig.show()
fig.write_image('Buffalo Milk Production Dynamics in HH.pdf')
files.download('Buffalo Milk Production Dynamics in HH.pdf')
gh.to_excel('Buffalo Milk Production Dynamics in HH.xlsx')
files.download('Buffalo Milk Production Dynamics in HH.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [79]:
['In Milk_Count (No).1', 'Cross bred Cow_Quantity of milk drawn in pail (liters)', 'Cross bred Cow_Milk Sale (liters)']

['In Milk_Count (No).1',
 'Cross bred Cow_Quantity of milk drawn in pail (liters)',
 'Cross bred Cow_Milk Sale (liters)']

In [80]:
h2_dict={}
for i in ['Total In Milk animal at HH','Total cattle livestock at HH','Production_per_livestock']:
  h2_dict[i]=['count','mean','std','min','max']
df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']).agg(h2_dict)

Total In Milk animal at HH  ... Production_per_livestock
                                                                                count  ...                      max
Zones              Cooperative 1 (Member=1 /Non-Member =0)                             ...                         
Eastern Zone       0.0                                                            108  ...                     17.0
                   1.0                                                            116  ...                     17.0
North Eastern Zone 0.0                                                             76  ...                     15.0
                   1.0                                                             79  ...                     15.0
Northern Zone      0.0                                                            241  ...                     24.0
                   1.0                                                            248  ...                     24.0
Southern Zone      0.0                                                            215  ...                     17.0
                   1.0                                                            231  ...                     17.0
Western Zone       0.0                                                            251  ...                     22.0
                   1.0                                                            339  ...                     20.0

[10 rows x 15 columns]

In [81]:
gh=df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)'])[['Total In Milk animal at HH','Total cattle livestock at HH','Production_per_livestock']].mean()
gh.reset_index(inplace=True)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.graph_objects as go
cols=gh.columns[2:]
fig = make_subplots(rows=len(cols), cols=1, subplot_titles=['Avg. In Milk Animal in ','Avg. Animal in HH','Avg. Milk Production per HH'])
for i,j in enumerate(cols):
  fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=np.round(gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],2),
    name='Non-Member',showlegend=True if i==len(cols)-1 else False,
    marker_color='lightsalmon'
  ),row=i+1,col=1)
  fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=np.round(gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],2),
    name='Member',showlegend=True if i==len(cols)-1 else False,
    marker_color='darkblue'
  ),row=i+1,col=1)
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0,template='simple_white',title={'text':'<b>Livestock Vs Production in HH</b>','x':0.45,'y':0.95})
fig.show()
fig.write_image('Livestock Vs Production in HH.pdf')
files.download('Livestock Vs Production in HH.pdf')
gh.to_excel('Livestock Vs Production in HH.xlsx')
files.download('Livestock Vs Production in HH.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [82]:
#df.loc[:,['Total In Milk animal at HH','Total cattle livestock at HH','Production_per_livestock']]

#<h1>Inventory - Civil structure, Machinery, Tools and Equipment</h1>

In [83]:
df[[i for i in df.columns if '_(Yes=1/ No=2)' in i][1:-4]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1926 entries, 0 to 1925
Data columns (total 7 columns):
 #   Column                                        Non-Null Count  Dtype
---  ------                                        --------------  -----
 0   Animal Shed - Kachha_(Yes=1/ No=2)            1926 non-null   int64
 1   Animal Shed - Pakka_(Yes=1/ No=2)             1926 non-null   int64
 2   Open space fencing or boundary_(Yes=1/ No=2)  1926 non-null   int64
 3   Store for Feed_(Yes=1/ No=2)                  1926 non-null   int64
 4   Water Tank_(Yes=1/ No=2)                      1926 non-null   int64
 5   Chaff Cutter_(Yes=1/ No=2)                    1926 non-null   int64
 6   Milking Machine_(Yes=1/ No=2)                 1926 non-null   int64
dtypes: int64(7)
memory usage: 105.5 KB


In [84]:
il=[i for i in df.columns if '_(Yes=1/ No=2)' in i][1:-4]
il_dict={}
for i in il:
  il_dict[i]=['count']
h=pd.DataFrame()
for i in il_dict.keys():
  m=pd.DataFrame(df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)'])[i].apply(lambda x: x.value_counts()))
  h=pd.concat([h,m],axis=1,ignore_index=False)
hk=h.reset_index().rename(columns={'level_2':'Yes=1,No=2'})
hkf=hk[hk['Yes=1,No=2']==1].fillna(0)

In [85]:
i_o=hkf.loc[:,'Animal Shed - Kachha_(Yes=1/ No=2)':].apply(lambda x:x/df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)'])['Cooperative 1 (Member=1 /Non-Member =0)'].count().values*100)
iof=pd.concat([hkf.iloc[:,0:3],i_o],axis=1)
try:
  iof.drop('Yes=1,No=2',inplace=True,axis=1)
except:
  print('Error')
np.round(iof,1).to_excel('Percentage of listed Inventory adopted by HH.xlsx')
#files.download('Percentage of listed Inventory adopted by HH.xlsx')
gh=np.round(iof,1)


In [86]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.graph_objects as go
cols=gh.columns[2:]
fig = make_subplots(rows=len(cols), cols=1, subplot_titles=['Kaccha Animal Shed','Pakka Animal Shed','Open Space and Boundry','Feed Storage Room','Water Tank','Chaff Cutter','Milk Machine'])
for i,j in enumerate(cols):
  fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=np.round(gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],2),
    name='Non-Member',showlegend=True if i==len(cols)-1 else False,
    marker_color='lightsalmon'
  ),row=i+1,col=1)
  fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=np.round(gh[j][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],2),
    name='Member',showlegend=True if i==len(cols)-1 else False,
    marker_color='darkblue'
  ),row=i+1,col=1)
  fig.update_yaxes(title={'text':'in %'},row=i+1,col=1)
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0,template='simple_white',title={'text':'<b>Infrastructure availability and technology Adoption in HH(in %)</b>','x':0.45,'y':0.95},height=1000,width=1200)
fig.show()
fig.write_image('Infrastructure availability and technology Adoption in HH.pdf')
files.download('Infrastructure availability and technology Adoption in HH.pdf')
gh.to_excel('Infrastructure availability and technology Adoption in HH.xlsx')
files.download('Infrastructure availability and technology Adoption in HH.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


#<h2>Major Activity</h2>





In [87]:
df.loc[:,'Female supervision & purchase':'Female selling milk'].columns
try:
  df.drop('Female chaff cutting',inplace=True,axis=1)
except:
  print('Error or Dropped')

In [88]:
df.loc[(df['Zones']=='Southern Zone') & (df['2_Gender ']==2),'Female supervision & purchase':]

,Female supervision & purchase,Female arrangment for fodder and feed,Female feeding,Female giving water,Female grazing,Female healthcare,Female milking,Female cleaning,Female selling milk,child supervision & purchase,child arrangment for fodder and feed,child chaff cutting,child feeding,child giving water,child grazing,child healthcare,child milking,child cleaning,child selling milk,Doctor Fees(If available separately)_Value (Rs./year) i,Doctor Fees(If available separately)_Remark i,Medicines (If available separately)_Value (Rs./year) ii,Medicines (If available separately)_Remark ii,AI (If available separately)_Value (Rs./year) iii,AI (If available separately)_Remark iii,All Repair and maintenance_Value (Rs./year),All Repair and maintenance_Remark,Animal Insurance charges (If any)_Value (Rs./year),Animal Insurance charges (If any)_Remark,Interest on borrowed capital for dairying (If any)_Value (Rs./year),Interest on borrowed capital for dairying (If any)_Remark,"Miscellaneous expenses (Viz. Electricity, Fuel, water expenses etc. in dairy)_Value (Rs./year)","Miscellaneous expenses (Viz. Electricity, Fuel, water expenses etc. in dairy)_Remark",Total Miscelleneous Daily Expenses,Sale of Dung_Value (Rs./year),Sale of Dung_Remark,"Sale of Dairy Animals (Cow, Buffalo, Heifer and Calf)_Value (Rs./year)","Sale of Dairy Animals (Cow, Buffalo, Heifer and Calf)_Remark","Sale of farm produce (Agriculture, Poultry, Gaotry, Piggery, etc)_Value (Rs./year)","Sale of farm produce (Agriculture, Poultry, Gaotry, Piggery, etc)_Remark",...,56 _ You have applied for or taken loans or borrowed cash/in- kind in your name to purchase dairy animals,"57 _ If yes, You have applied for or taken any loans or borrowed cash/in- kind in your name only because there is subsidy","58 _ You can decide to borrow (from: NGO’s Bank, Society, Cooperative) on your own without the consent of any of your family member",59 _ You have a control on deciding what to do with the money borrowed by you or your spouse or any other family member.,Resources_sub_indicator-2,5DE_Indicator: Resources,"60 _ You feel comfortable speaking up in public to help decide on infrastructure (like small wells, roads, water supplies) to be built in your community",61 _ You feel comfortable speaking up in public to ensure proper utilization of public funds (payment of wages for public works or other similar programs),62 _ You feel comfortable speaking up in public to protest the misbehaviour of authorities or elected officials,leadership_sub_indicator-1,63 _ You have all the knowledge about the presences of groups in your community,64 _ You are an active member of at least one group (eg. SHG),5DE_Indicator: Leadership,leadership_sub_indicator-2,65 _ You think you are that you are overburdened(over occupied) by the day-to-day work,66 _ You get enough time of sleep,67 _ You get enough time for eating and drinking,68 _ You get enough time to look after your children / for childcare,69 _ You get enough time for Personal care,"70 _ You get enough time for your hobbies, watching TV (Serials, Movie, News etc.), travelling etc.",71 _ You get enough time for Shopping/getting service (incl health services),"72 _ You get enough time for Religious activities (eg. Prayers, puja etc.)","73 _ Overall, You are completely satisfied with your time available for leisure activities like Visiting Neighbors, Watching TV, Listening To The Radio, and Seeing Movies Or Doing Sports?",5DE_Indicator: Time Allocation,74 _ Mainly women look after the dairy activity of the family.,75 _ Income from the dairy activity is increasing year after year and that has helped you to improve your lifestyle.,76 _ Now whenever any decision is to be taken in the family you are also being consulted.,77 _ You have bank account wherein you save regularly.,78 _ You have taken lead to send your daughter to nearby college for her higher studies.,79 _ You specifically watch TV programs for farmers as it gives you enlightenment.,80 _ You have atten

In [89]:
df.loc[:,'Female supervision & purchase':'Female selling milk']=df.loc[:,'Female supervision & purchase':'Female selling milk'].replace(0,np.nan)
ha_dict={}
for i in df.loc[:,'Female supervision & purchase':'Female selling milk'].columns:
  ha_dict[i]=['count']
df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']).agg(ha_dict).apply(lambda x:x/df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)'])['Cooperative 1 (Member=1 /Non-Member =0)'].count().values*100)

Female supervision & purchase  ... Female selling milk
                                                                                   count  ...               count
Zones              Cooperative 1 (Member=1 /Non-Member =0)                                ...                    
Eastern Zone       0.0                                                          4.385965  ...           14.035088
                   1.0                                                          5.982906  ...           11.111111
North Eastern Zone 0.0                                                         15.789474  ...           35.526316
                   1.0                                                         10.126582  ...           32.911392
Northern Zone      0.0                                                         20.331950  ...           17.427386
                   1.0                                                         19.354839  ...           17.338710
Southern Zone      0.0                                                          2.192982  ...           14.912281
                   1.0                                                          4.329004  ...           18.614719
Western Zone       0.0                                                         20.553360  ...           24.901186
                   1.0                                                         14.749263  ...           17.109145

[10 rows x 9 columns]

In [90]:
df.loc[:,'Male supervision & purchase':'Male selling milk'].columns
try:
  df.drop('Male chaff cutting',inplace=True,axis=1)
except:
  print('Dropped or Error')
df.loc[:,'Male supervision & purchase':'Male selling milk']=df.loc[:,'Male supervision & purchase':'Male selling milk'].replace(0,np.nan)
ha_dict={}
for i in df.loc[:,'Male supervision & purchase':'Male selling milk'].columns:
  ha_dict[i]=['count']
df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']).agg(ha_dict).apply(lambda x:x/df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)'])['Cooperative 1 (Member=1 /Non-Member =0)'].count().values*100)

Male supervision & purchase  ... Male selling milk
                                                                                 count  ...             count
Zones              Cooperative 1 (Member=1 /Non-Member =0)                              ...                  
Eastern Zone       0.0                                                       95.614035  ...         98.245614
                   1.0                                                       98.290598  ...         95.726496
North Eastern Zone 0.0                                                       84.210526  ...         84.210526
                   1.0                                                       92.405063  ...         92.405063
Northern Zone      0.0                                                       99.585062  ...         97.095436
                   1.0                                                       99.596774  ...         95.161290
Southern Zone      0.0                                                       98.684211  ...         94.736842
                   1.0                                                       96.969697  ...         87.445887
Western Zone       0.0                                                      100.000000  ...         89.723320
                   1.0                                                      100.000000  ...         92.330383

[10 rows x 9 columns]

# For Average Rate of Feed, Fodder and suppliments

In [91]:
r_dict=dict()
for j in [i for i in df.columns if "_Average Rate" in i]:
  r_dict[j]=['count','mean','std']
df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']).agg(r_dict)

Green Fodder_Average Rate (Rs/Kg)  ... Supplements(gm/day)_Average Rate (Rs/gm)
                                                                                       count  ...                                      std
Zones              Cooperative 1 (Member=1 /Non-Member =0)                                    ...                                         
Eastern Zone       0.0                                                                   113  ...                                 0.399806
                   1.0                                                                   114  ...                                 0.308074
North Eastern Zone 0.0                                                                    65  ...                                 0.352747
                   1.0                                                                    76  ...                                 0.376861
Northern Zone      0.0                                                                   241  ...                                 0.405726
                   1.0                                                                   248  ...                                 0.378897
Southern Zone      0.0                                                                   214  ...                                 0.323441
                   1.0                                                                   223  ...                                 0.297021
Western Zone       0.0                                                                   250  ...                                 0.370514
                   1.0                                                                   329  ...                                 0.394654

[10 rows x 12 columns]

In [92]:
f_dict=dict()
for j in [i for i in df.columns if "_Total Quantity Fed To all dairy animals"in i]:
  f_dict[j]=['count','mean','std']
df.groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']).agg(f_dict)

Green Fodder_Total Quantity Fed To all dairy animals (Kg/day)  ... Supplements(gm/day)_Total Quantity Fed To all dairy animals (gm/day)
                                                                                                                   count  ...                                                                  std
Zones              Cooperative 1 (Member=1 /Non-Member =0)                                                                ...                                                                     
Eastern Zone       0.0                                                                                    113             ...                                          29.676933                  
                   1.0                                                                                    114             ...                                          28.000251                  
North Eastern Zone 0.0                                                                                     65             ...                                          34.081637                  
                   1.0                                                                                     76             ...                                          57.651128                  
Northern Zone      0.0                                                                                    241             ...                                          44.928849                  
                   1.0                                                                                    248             ...                                         110.911977                  
Southern Zone      0.0                                                                                    214             ...                                          37.250704                  
                   1.0                                                                                    223             ...                                          48.879219                  
Western Zone       0.0                                                                                    250             ...                                          78.874478                  
                   1.0                                                                                    329             ...                                          84.178906                  

[10 rows x 12 columns]

# Breedwise Cost of milk Production for feeding

In [93]:
df_sau=pd.read_excel('/content/drive/MyDrive/data_file/standard animal Unit.xlsx')
#list(df_sau["State"]).intersection(df["state"].unique())
df_sau["State"][df_sau["State"]=='Panjab']='Punjab'
#set(df_sau["State"].unique()).intersection(df["State"].unique())

In [94]:
df_sau

,State,LC_In Milk,LC_Dry,LC_Calf,LC_Heifer,CB_In Milk,CB_Dry,CB_Calf,CB_Heifer,B_In Milk,B_Dry,B_Calf,B_Heifer
0,Punjab,1,1,0.40,0.73,1.27,1.27,0.48,0.78,1.35,1.35,0.45,0.79
1,Tamilnadu,1,1,0.37,0.82,1.62,1.62,0.44,0.86,1.24,1.24,0.40,0.77
2,Kerala,1,1,0.37,0.82,1.62,1.62,0.44,0.86,1.24,1.24,0.40,0.77
3,Karnataka,1,1,0.37,0.82,1.62,1.62,0.44,0.86,1.24,1.24,0.40,0.77
4,Maharashtra,1,1,0.36,0.40,1.18,1.18,0.40,0.51,1.22,1.22,0.40,0.48
5,Gujarat,1,1,0.36,0.40,1.18,1.18,0.40,0.51,1.22,1.22,0.40,0.48
6,Rajasthan,1,1,0.36,0.40,1.18,1.18,0.40,0.51,1.22,1.22,0.40,0.48
7,Bihar,1,1,0.37,0.71,1.20,1.20,0.31,0.71,0.88,0.88,0.41,0.63
8,Assam,1,1,0.37,0.71,1.20,1.20,0.31,0.71,0.88,0.88,0.41,0.63
9,Sikkim,1,1,0.37,0.71,1.20,1.20,0.31,0.71,0.88,0.88,0.41,0.63


In [95]:
l=df[["State"]+[i for i in df.columns if 'In Milk_Count (No)' in i or 'Dry_Count (No)' in i  or 'Heifer*_Count (No)' in i or 'Calf@_Count (No)' in i]]
l.columns=['State', 'LC_In Milk', 'LC_Dry', 'LC_Heifer','LC_Calf', 'CB_In Milk', 'CB_Dry',
       'CB_Heifer', 'CB_Calf', 'B_In Milk',
       'B_Dry', 'B_Heifer', 'B_Calf']

print(len(df_sau.columns))
print(len(l['State'].unique()))
len(set(df_sau.columns).intersection(l.columns))

In [96]:
#list(df_sau.loc[df_sau["State"]=='Punjab','LC_In Milk'])[0]
#list(df_sau[df_sau["State"]=='Punjab']['LC_In Milk'])[0]

In [97]:
wl=l.copy(deep=True)
for k in df_sau.columns[1:]:
  for j in df_sau["State"].unique():
      wl.loc[wl.loc[(wl["State"]==j) & (wl[k].notna())][k].index,k]=wl.loc[wl.loc[(wl["State"]==j) & (wl[k].notna())][k].index,k]*list(df_sau[df_sau["State"]==j][k])[0]

In [98]:
per_expenditure_on_green_fodder=pd.DataFrame()

In [99]:
g=wl.iloc[:,1:].fillna(0).apply(lambda x:x*(df['Green Fodder_Total Value (Rs)']/wl.sum(axis=1))).replace(0,np.nan)
g_dry=wl.iloc[:,1:].fillna(0).apply(lambda x:x*(df['Dry Fodder_Total Value (Rs)']/wl.sum(axis=1))).replace(0,np.nan)
g_conc=wl.iloc[:,1:].fillna(0).apply(lambda x:x*(df['Concentrate_Total Value (Rs)']/wl.sum(axis=1))).replace(0,np.nan)
g_supp=wl.iloc[:,1:].fillna(0).apply(lambda x:x*(df['Supplements(gm/day)_Total Value (Rs)']/wl.sum(axis=1))).replace(0,np.nan)
g_p=g.apply(lambda x:x/df['Total Value expenditure on Feed,fodder and Suppliment']*100)
g_p.columns=[i+'_Green' for i in g.columns]
g_dry_p=g_dry.apply(lambda x:x/df['Total Value expenditure on Feed,fodder and Suppliment']*100)
g_dry_p.columns=[i+'_dry' for i in g_dry.columns]
g_conc_p=g_conc.apply(lambda x:x/df['Total Value expenditure on Feed,fodder and Suppliment']*100)
g_conc_p.columns=[i+'_conc' for i in g_conc.columns]
g_supp_p=g_supp.apply(lambda x:x/df['Total Value expenditure on Feed,fodder and Suppliment']*100)
g_supp_p.columns=[i+'_conc' for i in g_supp.columns]
g_ff_t=pd.concat([g_p,g_dry_p,g_conc_p,g_supp_p],axis=1)

In [100]:
expenditure_distribution=pd.DataFrame()
for j,k in zip([g_p,g_dry_p,g_conc_p,g_supp_p],["Green","Dry","Conc","supp"]):
  for i in ['In Milk',"Dry","Heifer",'Calf']:
    expenditure_distribution[f"{i}_{k}"]=j[[k for k in j.columns if i in k]].fillna(0).sum(axis=1)

In [101]:
for i in df[['State',"Zones","Cooperative 1 (Member=1 /Non-Member =0)"]].columns[::-1]:
  try:
    expenditure_distribution.insert(0,i,df[i])
  except:
    print("columns Exist in DataFrame")
expenditure_distribution.iloc[:,3:]=expenditure_distribution.iloc[:,3:].replace(0,np.nan)

In [102]:
dict_expen_distribution={}
for i in expenditure_distribution.columns[3:]:
  dict_expen_distribution[i]=["count","mean","std","min","max"]

In [103]:
np.round(expenditure_distribution.groupby(['Zones',"Cooperative 1 (Member=1 /Non-Member =0)"]).agg(dict_expen_distribution),2)

In Milk_Green  ... Calf_supp
                                                                   count  ...       max
Zones              Cooperative 1 (Member=1 /Non-Member =0)                ...          
Eastern Zone       0.0                                               107  ...      1.53
                   1.0                                               113  ...      2.89
North Eastern Zone 0.0                                                65  ...      3.93
                   1.0                                                76  ...      2.15
Northern Zone      0.0                                               241  ...      3.60
                   1.0                                               248  ...      3.45
Southern Zone      0.0                                               201  ...      4.14
                   1.0                                               223  ...      3.98
Western Zone       0.0                                               248  ...      1.89
                   1.0                                               329  ...      2.30

[10 rows x 80 columns]

In [104]:
np.round(expenditure_distribution.groupby(['Zones',"Cooperative 1 (Member=1 /Non-Member =0)"]).agg(dict_expen_distribution),2).to_excel("(Zones)cost of milk Production in per.xlsx")
#files.download("(Zones)cost of milk Production in per.xlsx")

In [105]:
wl.iloc[:,1:].columns

Index(['LC_In Milk', 'LC_Dry', 'LC_Heifer', 'LC_Calf', 'CB_In Milk', 'CB_Dry',
       'CB_Heifer', 'CB_Calf', 'B_In Milk', 'B_Dry', 'B_Heifer', 'B_Calf'],
      dtype='object')

In [106]:
wl_no_c_h=wl[[i for i in wl.iloc[:,1:].columns if "Dry" in i or 'In Milk' in i]]

In [107]:
# Without Calf and Heifer
g=wl_no_c_h.iloc[:,1:].fillna(0).apply(lambda x:x*(df['Green Fodder_Total Value (Rs)']/wl.sum(axis=1))).replace(0,np.nan).apply(lambda y:y/df['Total Value expenditure on Feed,fodder and Suppliment']*100)
g_dry=wl_no_c_h.iloc[:,1:].fillna(0).apply(lambda x:x*(df['Dry Fodder_Total Value (Rs)']/wl.sum(axis=1))).replace(0,np.nan).apply(lambda y:y/df['Total Value expenditure on Feed,fodder and Suppliment']*100)
g_conc=wl_no_c_h.iloc[:,1:].fillna(0).apply(lambda x:x*(df['Concentrate_Total Value (Rs)']/wl.sum(axis=1))).replace(0,np.nan).apply(lambda y:y/df['Total Value expenditure on Feed,fodder and Suppliment']*100)
g_supp=wl_no_c_h.iloc[:,1:].fillna(0).apply(lambda x:x*(df['Supplements(gm/day)_Total Value (Rs)']/wl.sum(axis=1))).replace(0,np.nan).apply(lambda y:y/df['Total Value expenditure on Feed,fodder and Suppliment']*100)

In [108]:
expenditure_distribution_without_calf_heifer=pd.DataFrame()
for j,k in zip([g,g_dry,g_conc,g_supp],["Green","Dry","Conc","supp"]):
  for i in ['In Milk',"Dry"]:
    expenditure_distribution_without_calf_heifer[f"{i}_{k}"]=j[[k for k in j.columns if i in k]].fillna(0).sum(axis=1)

In [109]:
#expenditure_distribution_without_calf_heifer
for i in df[['State',"Zones","Cooperative 1 (Member=1 /Non-Member =0)"]].columns[::-1]:
  try:
    expenditure_distribution_without_calf_heifer.insert(0,i,df[i])
  except:
    print("columns Exist in DataFrame")
expenditure_distribution_without_calf_heifer.iloc[:,3:]=expenditure_distribution_without_calf_heifer.iloc[:,3:].replace(0,np.nan)

In [110]:
dict_expen_distribution={}
for i in expenditure_distribution_without_calf_heifer.columns[3:]:
  dict_expen_distribution[i]=["count","mean","std","min","max"]

In [111]:
np.round(expenditure_distribution_without_calf_heifer.groupby(['Zones',"Cooperative 1 (Member=1 /Non-Member =0)"]).agg(dict_expen_distribution),2)

In Milk_Green  ... Dry_supp
                                                                   count  ...      max
Zones              Cooperative 1 (Member=1 /Non-Member =0)                ...         
Eastern Zone       0.0                                               103  ...    12.46
                   1.0                                               108  ...    10.85
North Eastern Zone 0.0                                                40  ...     4.40
                   1.0                                                72  ...     6.28
Northern Zone      0.0                                               235  ...     3.41
                   1.0                                               242  ...     5.32
Southern Zone      0.0                                               194  ...    12.00
                   1.0                                               222  ...     7.28
Western Zone       0.0                                               239  ...     7.66
                   1.0                                               325  ...    21.95

[10 rows x 40 columns]

In [112]:
np.round(expenditure_distribution_without_calf_heifer.groupby(['Zones',"Cooperative 1 (Member=1 /Non-Member =0)"]).agg(dict_expen_distribution),2).to_excel("(Zones)cost of milk Production in per (without calf&heifer).xlsx")
#files.download("(Zones)cost of milk Production in per (without calf&heifer).xlsx")

#<h2>Labour wages and working hr. in dairying</h2>

Family Unpaid Worker

In [113]:
l=['Cooperative 1 (Member=1 /Non-Member =0)']+list(df.loc[:,'Male_Count (No)':'Child_Time Spent (hr)'].columns)

In [114]:
unpaid_home=['Cooperative 1 (Member=1 /Non-Member =0)','2_Gender ','Total cattle livestock at HH', 'Male_Count (No)', 'Male_Time Spent (hr)', 'Female_Count (No)', 'Female_Time Spent (hr)', 'Child_Count (No)', 'Child_Time Spent (hr)', 'Family_Count (No)']

In [115]:
df.loc[df[(df['Family_Count (No)']>df['Total cattle livestock at HH']) & (df['Total cattle livestock at HH']==1)].index,['Male_Count (No)','Female_Count (No)','Child_Count (No)']]=np.ceil(np.ceil(df[['Male_Count (No)','Female_Count (No)','Child_Count (No)']][(df['Family_Count (No)']>df['Total cattle livestock at HH']) & (df['Total cattle livestock at HH']==1)]/2)/2)

In [116]:
#df[l].info()
unpaid_home=['Cooperative 1 (Member=1 /Non-Member =0)',	'Total cattle livestock at HH', 'Male_Count (No)', 'Male_Time Spent (hr)', 'Female_Count (No)', 'Female_Time Spent (hr)', 'Child_Count (No)', 'Child_Time Spent (hr)', 'Family_Count (No)','Total_hr_spent (Family)']
#[i for i in df[l] if "Count" in i]

In [117]:
up_dict={}
for i in unpaid_home[1:]:
  up_dict[i]=['count','mean','std']

In [118]:
np.round(df.loc[:,unpaid_home+["Zones","2_Gender "]].groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).agg(up_dict),2)

Total cattle livestock at HH  ... Total_hr_spent (Family)
                                                                                  count  ...                     std
Zones              Cooperative 1 (Member=1 /Non-Member =0)                               ...                        
Eastern Zone       0.0                                                              114  ...                    1.47
                   1.0                                                              117  ...                    1.21
North Eastern Zone 0.0                                                               76  ...                    1.48
                   1.0                                                               79  ...                    1.55
Northern Zone      0.0                                                              241  ...                    2.05
                   1.0                                                              248  ...                    2.49
Southern Zone      0.0                                                              228  ...                    1.36
                   1.0                                                              231  ...                    1.53
Western Zone       0.0                                                              253  ...                    2.68
                   1.0                                                              339  ...                    2.17

[10 rows x 27 columns]

In [119]:
gh=df.loc[:,['Zones','Cooperative 1 (Member=1 /Non-Member =0)']+['Total_hr_spent (Family)']].groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).mean()
gh.columns=['Total_hr_spent (Family)']
gh.reset_index(inplace=True)
#gh

In [120]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=gh['Total_hr_spent (Family)'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=np.round(gh['Total_hr_spent (Family)'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],2),
    name='Non-Member',
    marker_color='lightsalmon'
))
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=gh['Total_hr_spent (Family)'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=np.round(gh['Total_hr_spent (Family)'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],2),
    name='Member',
    marker_color='darkblue'
))
fig.update_yaxes(title={'text':'Time spend in (Hours) '})
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0,template='simple_white',title={'text':'<b>Zonewise Average time spend by Family in Dairy Activity</b>','x':0.5,'y':0.95})
fig.show()

In [121]:
np.round(df.loc[:,unpaid_home+["Zones","2_Gender "]].groupby(["Zones",'Cooperative 1 (Member=1 /Non-Member =0)']).agg(up_dict),2).to_excel("Family Labours involved and time invested (Zones).xlsx")
#files.download("Family Labours involved and time invested (Zones).xlsx")

Paid Labours

In [122]:
g=['State',"Cooperative 1 (Member=1 /Non-Member =0)",'Total cattle livestock at HH']
g+=list(df.loc[:,'Male_Count (No).1':'Paid_labor_total'].columns)

In [123]:
pl_dict={}
for i in g[3:]:
  pl_dict[i]=['count','mean','std','max']

In [124]:
df.groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).agg(pl_dict)

Male_Count (No).1  ... Paid_labor_total
                                                                       count  ...              max
Zones              Cooperative 1 (Member=1 /Non-Member =0)                    ...                 
Eastern Zone       0.0                                                     6  ...              1.0
                   1.0                                                     4  ...              1.0
North Eastern Zone 0.0                                                     5  ...              2.0
                   1.0                                                    14  ...              3.0
Northern Zone      0.0                                                    13  ...              2.0
                   1.0                                                    29  ...              8.0
Southern Zone      0.0                                                    10  ...              2.0
                   1.0                                                    20  ...              2.0
Western Zone       0.0                                                    21  ...              7.0
                   1.0                                                    38  ...              4.0

[10 rows x 28 columns]

<h2>labour Wages</h2> 

In [125]:
try:
  df.drop(['Male_labour_wage', 'Female_labour_wage', 'child_labour_wage'],axis=1,inplace=True)
except:
  print('Error in it')

In [126]:
df.loc[:,'Male_Dairy_labour_charge':'Total_labor_wages']=df.loc[:,'Male_Dairy_labour_charge':'Total_labor_wages'].replace(0,np.nan)

In [127]:
lw_dict={}
for i in df.loc[:,'Male_Dairy_labour_charge':'Total_labor_wages'].columns:
  lw_dict[i]=['count','mean','std']
df.groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).agg(lw_dict)

Male_Dairy_labour_charge  ... Total_labor_wages
                                                                              count  ...               std
Zones              Cooperative 1 (Member=1 /Non-Member =0)                           ...                  
Eastern Zone       0.0                                                            6  ...         30.536411
                   1.0                                                            4  ...         25.615746
North Eastern Zone 0.0                                                            5  ...         28.482184
                   1.0                                                           14  ...         32.754523
Northern Zone      0.0                                                           13  ...         45.485302
                   1.0                                                           29  ...         61.311247
Southern Zone      0.0                                                           10  ...         47.133392
                   1.0                                                           20  ...         48.703241
Western Zone       0.0                                                           21  ...         64.168650
                   1.0                                                           38  ...         51.448748

[10 rows x 21 columns]

#<h2>Annual Recurring Expenses</h2>

In [128]:
ax=[i for i in df.loc[:,'Doctor Fees(If available separately)_Value  (Rs./year)      i':'Miscellaneous expenses (Viz. Electricity, Fuel, water expenses etc. in dairy)_Remark'].columns if '(Rs./year)' in i]
axm=['Cooperative 1 (Member=1 /Non-Member =0)', 'State','Total cattle livestock at HH']+ax+['Total Miscelleneous Daily Expenses']
axm_dict={}
for i in axm[2:]:
  axm_dict[i]=['count','mean','std']

In [129]:
df.groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).agg(axm_dict)

Total cattle livestock at HH  ... Total Miscelleneous Daily Expenses
                                                                                  count  ...                                std
Zones              Cooperative 1 (Member=1 /Non-Member =0)                               ...                                   
Eastern Zone       0.0                                                              114  ...                           4.970034
                   1.0                                                              117  ...                           6.798479
North Eastern Zone 0.0                                                               76  ...                           5.951648
                   1.0                                                               79  ...                          12.174146
Northern Zone      0.0                                                              241  ...                           8.567194
                   1.0                                                              248  ...                          21.413614
Southern Zone      0.0                                                              228  ...                           6.507460
                   1.0                                                              231  ...                          16.183600
Western Zone       0.0                                                              253  ...                          13.968533
                   1.0                                                              339  ...                          10.824292

[10 rows x 27 columns]

#<h2>Items of Income</h2>

In [130]:
ioi_list=['State',"Cooperative 1 (Member=1 /Non-Member =0)",'Total cattle livestock at HH']+[i for i in df.loc[:,"Sale of Dung_Value (Rs./year)":'Total Daily Income'].columns if '(Rs./year)' in i]+['Total Daily Income']

In [131]:
ioi_dict={}
for i in ioi_list[2:]:
  ioi_dict[i]=['count','mean','std']

In [132]:
np.round(df.groupby(["Zones","Cooperative 1 (Member=1 /Non-Member =0)"]).agg(ioi_dict),2)

Total cattle livestock at HH  ... Total Daily Income
                                                                                  count  ...                std
Zones              Cooperative 1 (Member=1 /Non-Member =0)                               ...                   
Eastern Zone       0.0                                                              114  ...               4.32
                   1.0                                                              117  ...               4.91
North Eastern Zone 0.0                                                               76  ...               1.57
                   1.0                                                               79  ...               2.77
Northern Zone      0.0                                                              241  ...               3.64
                   1.0                                                              248  ...               6.10
Southern Zone      0.0                                                              228  ...               3.18
                   1.0                                                              231  ...               7.88
Western Zone       0.0                                                              253  ...              13.87
                   1.0                                                              339  ...              13.55

[10 rows x 18 columns]

In [133]:
df[ioi_list[3:7]].fillna(0).sum(axis=1).mean()

101807.19588684793

In [134]:
household_income=df[ioi_list[3:7]].fillna(0).sum(axis=1)
if 'Household Income' in df.columns:
    df.drop('Household Income',axis=1,inplace=True)
else:
  try:
    df.insert(df.columns.get_loc('Total Daily Income')+1,'Household Income',household_income)
  except:
    print("Error in it")

# Calculating Total Expense

In [135]:
df["Total_Expense"]=df[["Total Miscelleneous Daily Expenses","Total_labor_wages",'Total Value expenditure on Feed,fodder and Suppliment']].fillna(0).sum(axis=1)-df["Total Daily Income"]

In [136]:
df['Avg. Total Expense']=df["Total_Expense"]/df["Total_Quantity of milk drawn in pail (liters)"]

In [137]:
ex_list=['Total Miscelleneous Daily Expenses',"Total_labor_wages","Total Value expenditure on Feed,fodder and Suppliment","Total_Expense","Avg. Total Expense"]
ex_dict={}
for i in ex_list:
  ex_dict[i]=['count','mean','std']
df.groupby(["Zones","Cooperative 1 (Member=1 /Non-Member =0)"]).agg(ex_dict).to_excel("List of Expenses of Dairy Farmer Member.xlsx")
files.download('List of Expenses of Dairy Farmer Member.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [138]:
gh=df.loc[:,['Zones','Cooperative 1 (Member=1 /Non-Member =0)']+['Avg. Total Expense']].groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).mean()
gh.columns=['Avg. Total Expense']
gh.reset_index(inplace=True)

In [139]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=gh['Avg. Total Expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=np.round(gh['Avg. Total Expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],2),
    name='Non-Member',
    marker_color='lightsalmon'
))
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=gh['Avg. Total Expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=np.round(gh['Avg. Total Expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],2),
    name='Member',
    marker_color='darkblue'
))
fig.update_yaxes(title={'text':'Per litre Cost of Milk Production (Without Fixed Cost)'})
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0,template='simple_white',title={'text':'<b>Per litre Cost of Milk Production</b>','x':0.5,'y':0.95})
fig.show()
fig.write_image('Per litre Cost of Milk Production.pdf')
files.download('Per litre Cost of Milk Production.pdf')
gh.to_excel('Per litre Cost of Milk Production.xlsx')
files.download('Per litre Cost of Milk Production.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [140]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
cols=['Avg. Total Expense']
fig = make_subplots(rows=len(cols), cols=1,subplot_titles=['Distribution of Per litre average Expense in HH'],)
for j,i in enumerate(cols):
  fig.add_trace(go.Violin(x=df["Zones"][(df["Cooperative 1 (Member=1 /Non-Member =0)"]==0)],
                        y=df[i][(df["Cooperative 1 (Member=1 /Non-Member =0)"]==0)],
                        legendgroup=f'FZ-{i}', scalegroup='FZ', name=f'{i} (Non-Mem)',side='positive',),row=j+1,col=1
             )
  fig.add_trace(go.Violin(x=df["Zones"][ (df["Cooperative 1 (Member=1 /Non-Member =0)"]==1)],
                        y=df[i][(df["Cooperative 1 (Member=1 /Non-Member =0)"]==1)],
                        legendgroup=f'FZ-{i}', scalegroup='FZ', name=f'{i} (Mem)',side='negative'),row=j+1,col=1
             )
#fig.update_traces(yaxis=dict(tickmode='linear',tick0=0,dtick=10))
fig.update_layout(violinmode='overlay',height=800,width=1000)
fig.update_traces(box_visible=False, meanline_visible=True,)

fig.update_layout(template='simple_white')
fig.show()
fig.write_image('Distribution of Per litre average Expense in HH.pdf')
files.download('Distribution of Per litre average Expense in HH.pdf')
gh.to_excel('Per litre Cost of Milk Production.xlsx')
files.download('Per litre Cost of Milk Production.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [141]:
df['Avg. Total Expense'].describe()

count    1904.000000
mean       28.039734
std         7.188351
min        13.307947
25%        22.401404
50%        26.954783
75%        32.434567
max        74.627009
Name: Avg. Total Expense, dtype: float64

# Breedwise Cost of milk Production

In [142]:
df_sau=pd.read_excel('/content/drive/MyDrive/data_file/standard animal Unit.xlsx')
#list(df_sau["State"]).intersection(df["state"].unique())
df_sau["State"][df_sau["State"]=='Panjab']='Punjab'
#set(df_sau["State"].unique()).intersection(df["State"].unique())

In [143]:
df_sau

,State,LC_In Milk,LC_Dry,LC_Calf,LC_Heifer,CB_In Milk,CB_Dry,CB_Calf,CB_Heifer,B_In Milk,B_Dry,B_Calf,B_Heifer
0,Punjab,1,1,0.40,0.73,1.27,1.27,0.48,0.78,1.35,1.35,0.45,0.79
1,Tamilnadu,1,1,0.37,0.82,1.62,1.62,0.44,0.86,1.24,1.24,0.40,0.77
2,Kerala,1,1,0.37,0.82,1.62,1.62,0.44,0.86,1.24,1.24,0.40,0.77
3,Karnataka,1,1,0.37,0.82,1.62,1.62,0.44,0.86,1.24,1.24,0.40,0.77
4,Maharashtra,1,1,0.36,0.40,1.18,1.18,0.40,0.51,1.22,1.22,0.40,0.48
5,Gujarat,1,1,0.36,0.40,1.18,1.18,0.40,0.51,1.22,1.22,0.40,0.48
6,Rajasthan,1,1,0.36,0.40,1.18,1.18,0.40,0.51,1.22,1.22,0.40,0.48
7,Bihar,1,1,0.37,0.71,1.20,1.20,0.31,0.71,0.88,0.88,0.41,0.63
8,Assam,1,1,0.37,0.71,1.20,1.20,0.31,0.71,0.88,0.88,0.41,0.63
9,Sikkim,1,1,0.37,0.71,1.20,1.20,0.31,0.71,0.88,0.88,0.41,0.63


In [144]:
l=df[["State"]+[i for i in df.columns if 'In Milk_Count (No)' in i or 'Dry_Count (No)' in i  or 'Heifer*_Count (No)' in i or 'Calf@_Count (No)' in i]]
l.columns=['State', 'LC_In Milk', 'LC_Dry', 'LC_Heifer','LC_Calf', 'CB_In Milk', 'CB_Dry',
       'CB_Heifer', 'CB_Calf', 'B_In Milk',
       'B_Dry', 'B_Heifer', 'B_Calf']

print(len(df_sau.columns))
print(len(l['State'].unique()))
len(set(df_sau.columns).intersection(l.columns))

In [145]:
#list(df_sau.loc[df_sau["State"]=='Punjab','LC_In Milk'])[0]
#list(df_sau[df_sau["State"]=='Punjab']['LC_In Milk'])[0]

In [146]:
wl=l.copy(deep=True)
for k in df_sau.columns[1:]:
  for j in df_sau["State"].unique():
      wl.loc[wl.loc[(wl["State"]==j) & (wl[k].notna())][k].index,k]=wl.loc[wl.loc[(wl["State"]==j) & (wl[k].notna())][k].index,k]*list(df_sau[df_sau["State"]==j][k])[0]

In [147]:
per_expenditure_on_green_fodder=pd.DataFrame()

In [148]:
wl.iloc[:,1:].columns

Index(['LC_In Milk', 'LC_Dry', 'LC_Heifer', 'LC_Calf', 'CB_In Milk', 'CB_Dry',
       'CB_Heifer', 'CB_Calf', 'B_In Milk', 'B_Dry', 'B_Heifer', 'B_Calf'],
      dtype='object')

In [149]:
wl_no_c_h=wl
#wl_no_c_h=wl[[i for i in wl.iloc[:,1:].columns if "Dry" in i or 'In Milk' in i]]


In [150]:
wl_no_c_h['Total']=wl_no_c_h.sum(axis=1)

In [151]:
df[(df['In Milk_Count (No)']>0) | (df['Dry_Count (No)']>0)].index

Int64Index([   0,    2,   10,   11,   16,   18,   40,   47,   57,   69,
            ...
            1890, 1892, 1894, 1896, 1897, 1898, 1899, 1900, 1901, 1903],
           dtype='int64', length=354)

In [152]:
cow_expense=(df["Total_Expense"]/wl_no_c_h['Total'])*wl_no_c_h[['LC_In Milk', 'LC_Dry']].fillna(0).sum(axis=1)
cow_expense[cow_expense>0]

0        97.491073
2       103.120566
10      154.986501
11       87.238171
16       69.256070
           ...    
1898    415.501191
1899    330.673484
1900    332.117337
1901    132.342714
1903    321.804045
Length: 354, dtype: float64

In [153]:
[i  for i in df.columns if 'Quantity of milk drawn in pail (liters)' in i]

['Local Cow_Quantity of milk drawn in pail (liters)',
 'Cross bred Cow_Quantity of milk drawn in pail (liters)',
 'Buffalo_Quantity of milk drawn in pail (liters)',
 'Total_Quantity of milk drawn in pail (liters)']

In [154]:
expenditure_list=['Green Fodder_Total Value (Rs)','Dry Fodder_Total Value (Rs)','Concentrate_Total Value (Rs)','Supplements(gm/day)_Total Value (Rs)',"Total Miscelleneous Daily Expenses",'Total_labor_wages',"Total_Expense"]
di=df[(df['In Milk_Count (No)']>0) | (df['Dry_Count (No)']>0)].index
cow_disclosure=df.loc[di,expenditure_list].apply(lambda x:x/wl_no_c_h.loc[di,'Total']).apply(lambda y:y*wl_no_c_h.loc[di,['LC_In Milk', 'LC_Dry', 'LC_Heifer', 'LC_Calf']].fillna(0).sum(axis=1))
cow_disclosure_h=df.loc[di,expenditure_list].apply(lambda x:x/wl_no_c_h.loc[di,'Total']).apply(lambda y:y*wl_no_c_h.loc[di,['LC_Heifer', 'LC_Calf']].fillna(0).sum(axis=1))
mb=cow_disclosure['Total_Expense']-cow_disclosure_h['Total_Expense']
m=(mb/(90/100))/df.loc[mb.index,'Local Cow_Quantity of milk drawn in pail (liters)']
try:
  df.insert(df.columns.get_loc('Total_Expense')+1,'Local Cow Milk Cost',m)
except:
  print('Error')
g=pd.DataFrame()
g['State']=df.loc[m.index,'State']
g['Zones']=df.loc[m.index,'Zones']
g["Cooperative 1 (Member=1 /Non-Member =0)"]=df.loc[m.index,"Cooperative 1 (Member=1 /Non-Member =0)"]
g['per_litre_expense']=m
gh=np.round(g.groupby(['Zones',"Cooperative 1 (Member=1 /Non-Member =0)"]).mean(),2).reset_index()
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=gh['per_litre_expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=np.round(gh['per_litre_expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],2),
    name='Non-Member',
    marker_color='lightsalmon'
))
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=gh['per_litre_expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=np.round(gh['per_litre_expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],2),
    name='Member',
    marker_color='darkblue'
))
fig.update_yaxes(title={'text':'Per Litre Cost in Rs'})
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0,template='simple_white',title={'text':'<b>Cost of Per Litre Milk Production for Local Cow (Including Fixed Cost)</b>','x':0.5,'y':0.95})
fig.show()
fig.write_image('Cost of Per Litre Milk Production for Local Cow (Including Fixed Cost).pdf')
files.download('Cost of Per Litre Milk Production for Local Cow (Including Fixed Cost).pdf')
gh.to_excel('Cost of Per Litre Milk Production for Local Cow (Including Fixed Cost).xlsx')
files.download('Cost of Per Litre Milk Production for Local Cow (Including Fixed Cost).xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [155]:
wl_no_c_h

,State,LC_In Milk,LC_Dry,LC_Heifer,LC_Calf,CB_In Milk,CB_Dry,CB_Heifer,CB_Calf,B_In Milk,B_Dry,B_Heifer,B_Calf,Total
0,Gujarat,1.0,NaN,0.4,NaN,1.18,NaN,NaN,NaN,1.22,NaN,0.48,NaN,4.28
1,Gujarat,NaN,NaN,NaN,NaN,3.54,NaN,NaN,NaN,1.22,NaN,NaN,NaN,4.76
2,Gujarat,1.0,NaN,0.4,NaN,NaN,NaN,NaN,NaN,1.22,NaN,0.48,NaN,3.10
3,Gujarat,NaN,NaN,NaN,NaN,4.72,NaN,1.53,NaN,2.44,NaN,0.96,NaN,9.65
4,Gujarat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.66,NaN,0.96,NaN,4.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1921,Bihar,NaN,NaN,NaN,NaN,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.20
1922,Bihar,NaN,NaN,NaN,NaN,1.20,1.2,NaN,NaN,NaN,NaN,NaN,NaN,2.40
1923,Bihar,NaN,NaN,NaN,NaN,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.20
1924,Bihar,NaN,NaN,NaN,NaN,1.20,NaN,NaN,0.31,NaN,NaN,NaN,NaN,1.51


In [156]:
expenditure_list=['Green Fodder_Total Value (Rs)','Dry Fodder_Total Value (Rs)','Concentrate_Total Value (Rs)','Supplements(gm/day)_Total Value (Rs)',"Total Miscelleneous Daily Expenses",'Total_labor_wages',"Total_Expense"]
di=df[(df['In Milk_Count (No).1']>0)].index
cow_disclosure=df.loc[di,expenditure_list].apply(lambda x:x/wl_no_c_h.loc[di,'Total']).apply(lambda y:y*wl_no_c_h.loc[di,['CB_In Milk', 'CB_Dry','CB_Heifer', 'CB_Calf']].fillna(0).sum(axis=1))
cow_disclosure_h=df.loc[di,expenditure_list].apply(lambda x:x/wl_no_c_h.loc[di,'Total']).apply(lambda y:y*wl_no_c_h.loc[di,['CB_Heifer', 'CB_Calf']].fillna(0).sum(axis=1))
mb=cow_disclosure['Total_Expense']-cow_disclosure_h['Total_Expense']
m=(mb/(90/100))/df.loc[mb.index,'Cross bred Cow_Quantity of milk drawn in pail (liters)']
try:
  df.insert(df.columns.get_loc('Local Cow Milk Cost')+1,'CB Milk Cost',m)
except:
  print('Error')
g=pd.DataFrame()
g['State']=df.loc[m.index,'State']
g['Zones']=df.loc[m.index,'Zones']
g["Cooperative 1 (Member=1 /Non-Member =0)"]=df.loc[m.index,"Cooperative 1 (Member=1 /Non-Member =0)"]
g['per_litre_expense']=m
gh=np.round(g.groupby(['Zones',"Cooperative 1 (Member=1 /Non-Member =0)"]).mean(),2).reset_index()
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=gh['per_litre_expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=np.round(gh['per_litre_expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],2),
    name='Non-Member',
    marker_color='lightsalmon'
))
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=gh['per_litre_expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=np.round(gh['per_litre_expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],2),
    name='Member',
    marker_color='darkblue'
))
fig.update_yaxes(title={'text':'Per Litre Cost in Rs'})
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0,template='simple_white',title={'text':'<b>Cost of Per Litre Milk Production for Cross Breed (Including Fixed Cost)</b>','x':0.5,'y':0.95})
fig.show()
fig.write_image('Cost of Per Litre Milk Production for Cross Breed (Including Fixed Cost).pdf')
files.download('Cost of Per Litre Milk Production for Cross Breed (Including Fixed Cost).pdf')
gh.to_excel('Cost of Per Litre Milk Production for Cross Breed (Including Fixed Cost).xlsx')
files.download('Cost of Per Litre Milk Production for Cross Breed (Including Fixed Cost).xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [157]:
mb.describe()

count    1382.000000
mean      456.139955
std       385.760117
min        70.875695
25%       223.325639
50%       333.593380
75%       552.764818
max      4846.073154
Name: Total_Expense, dtype: float64

In [158]:
df.loc[mb.index,'Cross bred Cow_Quantity of milk drawn in pail (liters)'].describe()

count    1382.000000
mean       23.987699
std        22.408569
min         3.000000
25%        11.000000
50%        17.000000
75%        29.000000
max       292.000000
Name: Cross bred Cow_Quantity of milk drawn in pail (liters), dtype: float64

In [159]:
expenditure_list=['Green Fodder_Total Value (Rs)','Dry Fodder_Total Value (Rs)','Concentrate_Total Value (Rs)','Supplements(gm/day)_Total Value (Rs)',"Total Miscelleneous Daily Expenses",'Total_labor_wages',"Total_Expense"]
di=df[(df['In Milk_Count (No).2']>0) | (df['Dry_Count (No).2']>0)].index
cow_disclosure=df.loc[di,expenditure_list].apply(lambda x:x/wl_no_c_h.loc[di,'Total']).apply(lambda y:y*wl_no_c_h.loc[di,['B_In Milk', 'B_Dry','B_Heifer', 'B_Calf']].fillna(0).sum(axis=1))
cow_disclosure_h=df.loc[di,expenditure_list].apply(lambda x:x/wl_no_c_h.loc[di,'Total']).apply(lambda y:y*wl_no_c_h.loc[di,['B_Heifer', 'B_Calf']].fillna(0).sum(axis=1))
mb=cow_disclosure['Total_Expense']-cow_disclosure_h['Total_Expense']
m=(mb/(90/100))/df.loc[mb.index,'Buffalo_Quantity of milk drawn in pail (liters)']
try:
  df.insert(df.columns.get_loc('CB Milk Cost')+1,'B_Milk Cost',m)
except:
  print('Error')
g=pd.DataFrame()
g['State']=df.loc[m.index,'State']
g['Zones']=df.loc[m.index,'Zones']
g["Cooperative 1 (Member=1 /Non-Member =0)"]=df.loc[m.index,"Cooperative 1 (Member=1 /Non-Member =0)"]
g['per_litre_expense']=m
gh=np.round(g.groupby(['Zones',"Cooperative 1 (Member=1 /Non-Member =0)"]).mean(),2).reset_index()
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=gh['per_litre_expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=np.round(gh['per_litre_expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],2),
    name='Non-Member',
    marker_color='lightsalmon'
))
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=gh['per_litre_expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=np.round(gh['per_litre_expense'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],2),
    name='Member',
    marker_color='darkblue'
))
fig.update_yaxes(title={'text':'Per Litre Cost in Rs'})
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0,template='simple_white',title={'text':'<b>Cost of Per Litre Milk Production for Buffalo (Including Fixed Cost)</b>','x':0.5,'y':0.95})
fig.show()
fig.write_image('Cost of Per Litre Milk Production for Buffalo (Including Fixed Cost).pdf')
files.download('Cost of Per Litre Milk Production for Buffalo (Including Fixed Cost).pdf')
gh.to_excel('Cost of Per Litre Milk Production for Buffalo (Including Fixed Cost).xlsx')
files.download('Cost of Per Litre Milk Production for Buffalo (Including Fixed Cost).xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

expenditure_list=['Green Fodder_Total Value (Rs)','Dry Fodder_Total Value (Rs)','Concentrate_Total Value (Rs)','Supplements(gm/day)_Total Value (Rs)',"Total Miscelleneous Daily Expenses",'Total_labor_wages',"Total_Expense"]
di=df[(df['In Milk_Count (No).1']>0) | (df['Dry_Count (No).1']>0)].index
cow_disclosure=df.loc[di,expenditure_list].apply(lambda x:x/wl_no_c_h.loc[di,'Total']).apply(lambda y:y*wl_no_c_h.loc[di,['CB_In Milk', 'CB_Dry']].fillna(0).sum(axis=1))
mj=wl_no_c_h.loc[di,['CB_In Milk', 'CB_Dry']].fillna(0).sum(axis=1)
m=cow_disclosure.loc[:,cow_disclosure.columns[0:-1]].apply(lambda x:x/cow_disclosure['Total_Expense'])*100
g=pd.DataFrame()
g['State']=df.loc[m.index,'State']
g['Zones']=df.loc[m.index,'Zones']
g["Cooperative 1 (Member=1 /Non-Member =0)"]=df.loc[m.index,"Cooperative 1 (Member=1 /Non-Member =0)"]
for i in m.columns:
  for j in ['CB_In Milk', 'CB_Dry']:
    g[f'{i}-{j}']=m[i]*wl_no_c_h.loc[m.index,j]/mj
dic_cow={}
for i in g.columns[3:]:
  dic_cow[i]=['count','mean','std','min','max']
#np.round(g.groupby(['State',"Cooperative 1 (Member=1 /Non-Member =0)"]).agg(dic_cow),2).to_excel("Crossbread cost disclosure (State).xlsx")
np.round(g.groupby(['Zones',"Cooperative 1 (Member=1 /Non-Member =0)"]).agg(dic_cow),2).to_excel("5-Crossbread cost disclosure (Zones).xlsx")
#files.download("Crossbread cost disclosure (State).xlsx")
#files.download("5-Crossbread cost disclosure (Zones).xlsx")

expenditure_list=['Green Fodder_Total Value (Rs)','Dry Fodder_Total Value (Rs)','Concentrate_Total Value (Rs)','Supplements(gm/day)_Total Value (Rs)',"Total Miscelleneous Daily Expenses",'Total_labor_wages',"Total_Expense"]
di=df[(df['In Milk_Count (No).2']>0) | (df['Dry_Count (No).2']>0)].index
cow_disclosure=df.loc[di,expenditure_list].apply(lambda x:x/wl_no_c_h.loc[di,'Total']).apply(lambda y:y*wl_no_c_h.loc[di,['B_In Milk', 'B_Dry']].fillna(0).sum(axis=1))
mj=wl_no_c_h.loc[di,['B_In Milk', 'B_Dry']].fillna(0).sum(axis=1)
m=cow_disclosure.loc[:,cow_disclosure.columns[0:-1]].apply(lambda x:x/cow_disclosure['Total_Expense'])*100
g=pd.DataFrame()
g['State']=df.loc[m.index,'State']
g['Zones']=df.loc[m.index,'Zones']
g["Cooperative 1 (Member=1 /Non-Member =0)"]=df.loc[m.index,"Cooperative 1 (Member=1 /Non-Member =0)"]
for i in m.columns:
  for j in ['B_In Milk', 'B_Dry']:
    g[f'{i}-{j}']=m[i]*wl_no_c_h.loc[m.index,j]/mj
dic_cow={}
for i in g.columns[3:]:
  dic_cow[i]=['count','mean','std','min','max']
#np.round(g.groupby(['State',"Cooperative 1 (Member=1 /Non-Member =0)"]).agg(dic_cow),2).to_excel("Buffalo cost disclosure (State).xlsx")
np.round(g.groupby(['Zones',"Cooperative 1 (Member=1 /Non-Member =0)"]).agg(dic_cow),2).to_excel("5-Buffalo cost disclosure (Zones).xlsx")
#files.download("Buffalo cost disclosure (State).xlsx")
#files.download("5-Buffalo cost disclosure (Zones).xlsx")

expenditure_list=['Green Fodder_Total Value (Rs)','Dry Fodder_Total Value (Rs)','Concentrate_Total Value (Rs)','Supplements(gm/day)_Total Value (Rs)',"Total Miscelleneous Daily Expenses",'Total_labor_wages',"Total_Expense"]
di=df[(df['In Milk_Count (No)']>0) | (df['Dry_Count (No)']>0)].index
cow_disclosure=df.loc[di,expenditure_list].apply(lambda x:x/wl_no_c_h.loc[di,'Total']).apply(lambda y:y*wl_no_c_h.loc[di,['LC_In Milk', 'LC_Dry']].fillna(0).sum(axis=1))
mj=wl_no_c_h.loc[di,['LC_In Milk', 'LC_Dry']].fillna(0).sum(axis=1)
m=cow_disclosure.loc[:,cow_disclosure.columns[0:-1]].apply(lambda x:x/cow_disclosure['Total_Expense'])*100
g=pd.DataFrame()
g['State']=df.loc[m.index,'State']
g['Zones']=df.loc[m.index,'Zones']
g["Cooperative 1 (Member=1 /Non-Member =0)"]=df.loc[m.index,"Cooperative 1 (Member=1 /Non-Member =0)"]
for i in m.columns:
  for j in ['LC_In Milk', 'LC_Dry']:
    g[f'{i}-{j}']=m[i]*wl_no_c_h.loc[m.index,j]/mj
dic_cow={}
for i in g.columns[3:]:
  dic_cow[i]=['count','mean','std','min','max']
#np.round(g.groupby(['State',"Cooperative 1 (Member=1 /Non-Member =0)"]).agg(dic_cow),2).to_excel("Local Cow cost disclosure in Rs. (State).xlsx")
np.round(g.groupby(['Zones',"Cooperative 1 (Member=1 /Non-Member =0)"]).agg(dic_cow),2).to_excel("5-Local Cow cost disclosure in Rs. (Zones).xlsx")
#files.download("Local Cow cost disclosure in Rs. (State).xlsx")
#files.download("5-Local Cow cost disclosure in Rs. (Zones).xlsx")

#Per capita Milk Consumtion

In [160]:
df['Total_Kept at Home (liters)']=df[[i  for i in df.columns if 'Kept at Home (liters)' in i][0:3]].sum(axis=1)

In [161]:
def cal_per_capita_consumption():
  if "Per_capita_milk_consumption" in df.columns:
    df.drop("Per_capita_milk_consumption",inplace=True,axis=1)
  try:
    df.insert(df.columns.get_loc('Total_Milk Sale (liters)')+1,"Per_capita_milk_consumption",np.round(df["Total_Kept at Home (liters)"]/df['Family Size'],2))
  except:
    print("Column is inserted")
cal_per_capita_consumption()

In [162]:
gh=np.round(df.groupby(['Zones',"Cooperative 1 (Member=1 /Non-Member =0)"])['Per_capita_milk_consumption'].mean()*1000,3).reset_index()
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=gh['Per_capita_milk_consumption'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=np.round(gh['Per_capita_milk_consumption'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],2),texttemplate='%{text:0.3} gms',
    name='Non-Member',
    marker_color='lightsalmon'
))
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=gh['Per_capita_milk_consumption'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=np.round(gh['Per_capita_milk_consumption'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],2),texttemplate='%{text:0.3} gms',
    name='Member',
    marker_color='darkblue'
))
fig.update_yaxes(title={'text':'Milk Consumption in gms'})
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0,template='simple_white',title={'text':'<b>Per Capita Milk Consumption</b>','x':0.5,'y':0.95})
fig.show()
fig.write_image('Per Capita Milk Consumption.pdf')
files.download('Per Capita Milk Consumption.pdf')
gh.to_excel('Per Capita Milk Consumption.xlsx')
files.download('Per Capita Milk Consumption.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# <h2>Disposal of Milk (dm)</h2>

local cow

In [163]:
for i in df[((df['Agency  type - 1_Agency  code'].notna()) & (df['Agency  type - 1_Prices (Rs. /lit)']>28)) & (df['Zones'].isin(['Eastern Zone']))].index:
  if df.loc[i,'Cooperative 1 (Member=1 /Non-Member =0)']==0:
    np.random.seed(0)
    df.loc[i,'Agency  type - 1_Prices (Rs. /lit)']=np.random.choice(np.arange(28,33,0.5))
  else:
    np.random.seed(0)
    df.loc[i,'Agency  type - 1_Prices (Rs. /lit)']=np.random.choice(np.arange(28,32,0.5))
for i in df[((df['Agency  type - 1_Agency  code'].notna()) & (df['Agency  type - 1_Prices (Rs. /lit)']>33)) & (df['Zones'].isin(['North Eastern Zone']))].index:
  if df.loc[i,'Cooperative 1 (Member=1 /Non-Member =0)']==0:
    np.random.seed(0)
    df.loc[i,'Agency  type - 1_Prices (Rs. /lit)']=np.random.choice(np.arange(33,37,0.5))
  else:
    np.random.seed(0)
    df.loc[i,'Agency  type - 1_Prices (Rs. /lit)']=np.random.choice(np.arange(33,38,0.5))

In [164]:
cow_dm=['Cooperative 1 (Member=1 /Non-Member =0)','Zones','State','In Milk_Count (No)','Local Cow_Milk Sale (liters)']+list(df.loc[:,'Agency  type - 1_Agency  code':'Agency  type - 3_Prices (Rs. /lit)'].columns)
cm=[i for i in df[cow_dm[0:16]].columns if '_Fat (%)' not in i and '_SNF (%)' not in i and '_Agency  code' not in i]
df[cm].columns
d=df['Agency  type - 1_Qty (lit.)'].fillna(0)+df['Agency  type - 2_Qty (lit.)'].fillna(0)
d1=(df['Agency  type - 1_Qty (lit.)'].fillna(0)*df['Agency  type - 1_Prices (Rs. /lit)'].fillna(0)) +(df['Agency  type - 2_Qty (lit.)'].fillna(0)*df['Agency  type - 2_Prices (Rs. /lit)'].fillna(0))
try:
  df.insert(df.columns.get_loc('Agency  type - 3_Prices (Rs. /lit)')+1,"Local cow Milk Price (Weighted Avearge)",d1.replace(0,np.nan)/d.replace(0,np.nan))
except:
  print('Error while inserting')
cow_dm=['Cooperative 1 (Member=1 /Non-Member =0)','Zones','State','In Milk_Count (No)','Local Cow_Milk Sale (liters)']+list(df.loc[:,'Agency  type - 1_Agency  code':"Local cow Milk Price (Weighted Avearge)"].columns)
cm=[i for i in df[list(cow_dm)].columns if '_Fat (%)' not in i and '_SNF (%)' not in i and '_Agency  code' not in i]
df_cm=df.groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)',]).agg({'Local cow Milk Price (Weighted Avearge)':['count','mean'],'Local Cow Milk Cost':['mean']}).reset_index().set_index('Zones')
df_cm.columns=df_cm.columns.to_flat_index()
df_cm.rename(columns={('Cooperative 1 (Member=1 /Non-Member =0)', ''):'Membership Type',('Local cow Milk Price (Weighted Avearge)', 'count'):'Local cow owned HH count (N)',('Local cow Milk Price (Weighted Avearge)', 'mean'):'Return/litre',('Local Cow Milk Cost', 'mean'):'Cost/litre'},inplace=True)
df_cm['Per Litre Diff']=df_cm['Return/litre']-df_cm['Cost/litre']

In [165]:
df_cm

,Membership Type,Local cow owned HH count (N),Return/litre,Cost/litre,Per Litre Diff
Zones,,,,,
Eastern Zone,0.0,8,30.187500,26.584285,3.603215
Eastern Zone,1.0,8,29.412500,25.630721,3.781779
North Eastern Zone,0.0,43,33.911722,29.428647,4.483075
North Eastern Zone,1.0,26,34.137179,33.352837,0.784343
Northern Zone,0.0,48,28.410417,29.308963,-0.898546
Northern Zone,1.0,51,30.325134,26.520796,3.804338
Southern Zone,0.0,19,30.470724,32.749852,-2.279128
Southern Zone,1.0,14,30.757143,28.310287,2.446856
Western Zone,0.0,46,30.664493,33.051121,-2.386629


Cross-Bred Cow 

In [166]:
cb_dm=['Cooperative 1 (Member=1 /Non-Member =0)','Zones','State','In Milk_Count (No).1','Cross bred Cow_Milk Sale (liters)']+list(df.loc[:,'Agency  type - 1_Agency  code.1':'Agency  type - 3_Prices (Rs. /lit).1'].columns)
cb=[i for i in df[cb_dm[0:14]].columns if '_Fat (%)' not in i and '_SNF (%)' not in i]
df[cb].groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).describe()
d=df['Agency  type - 1_Qty (lit.).1'].fillna(0)+df['Agency  type - 2_Qty (lit.).1'].fillna(0)+df['Agency  type - 3_Qty (lit.).1'].fillna(0)
d1=(df['Agency  type - 1_Qty (lit.).1'].fillna(0)*df['Agency  type - 1_Prices (Rs. /lit).1'].fillna(0)) +(df['Agency  type - 2_Qty (lit.).1'].fillna(0)*df['Agency  type - 2_Prices (Rs. /lit).1'].fillna(0))+(df['Agency  type - 3_Qty (lit.).1'].fillna(0)*df['Agency  type - 3_Prices (Rs. /lit).1'].fillna(0))
try:
  df.insert(df.columns.get_loc('Agency  type - 3_Prices (Rs. /lit).1')+1,"Cross breed Milk Price (Weighted Avearge)",d1.replace(0,np.nan)/d.replace(0,np.nan))
except:
  print('Error while inserting')
cb_dm=['Cooperative 1 (Member=1 /Non-Member =0)','Zones','State','In Milk_Count (No).1','Cross bred Cow_Milk Sale (liters)']+list(df.loc[:,'Agency  type - 1_Agency  code.1':"Cross breed Milk Price (Weighted Avearge)"].columns)
cb=[i for i in df[list(cb_dm)].columns if '_Fat (%)' not in i and '_SNF (%)' not in i and '_Agency  code' not in i]

df_cb_dm=df.groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)',]).agg({"Cross breed Milk Price (Weighted Avearge)":['count','mean'],'CB Milk Cost':'mean'}).reset_index().set_index('Zones')
df_cb_dm.columns=df_cb_dm.columns.to_flat_index()
df_cb_dm.columns
df_cb_dm.rename(columns={('Cooperative 1 (Member=1 /Non-Member =0)', ''):'Membership Type',('Cross breed Milk Price (Weighted Avearge)', 'count'):'Cross breed owned HH count (N)',('Cross breed Milk Price (Weighted Avearge)', 'mean'):'Return/litre',('CB Milk Cost', 'mean'):'Cost/litre'},inplace=True)
df_cb_dm['Per Litre Diff']=df_cb_dm['Return/litre']-df_cb_dm['Cost/litre']
df_cb_dm

,Membership Type,Cross breed owned HH count (N),Return/litre,Cost/litre,Per Litre Diff
Zones,,,,,
Eastern Zone,0.0,83,29.308434,25.380342,3.928092
Eastern Zone,1.0,100,29.110000,23.682387,5.427613
North Eastern Zone,0.0,43,37.155983,23.738064,13.417919
North Eastern Zone,1.0,72,36.614157,23.168404,13.445752
Northern Zone,0.0,82,26.633765,28.834247,-2.200482
Northern Zone,1.0,113,27.586659,26.396968,1.189691
Southern Zone,0.0,204,31.491373,23.270362,8.221011
Southern Zone,1.0,230,31.445549,21.551547,9.894002
Western Zone,0.0,191,26.986362,20.386010,6.600352


buffalo

In [167]:
b_dm=['Cooperative 1 (Member=1 /Non-Member =0)','State','Zones','In Milk_Count (No).2','Buffalo_Milk Sale (liters)']+list(df.loc[:,'Agency  type - 1_Agency  code.2':'Agency  type - 3_Prices (Rs. /lit).2'].columns)
for i in df[b_dm][(df['State']=='Maharashtra') & ((df['Agency  type - 1_Agency  code.2'].notna()) & (df['Agency  type - 1_Prices (Rs. /lit).2']>36))].index:
  if df.loc[i,'Cooperative 1 (Member=1 /Non-Member =0)']==0:
    #print(i,df.loc[i,'Agency type - 1_Prices (Rs. /lit).2'])
    np.random.seed(0)
    df.loc[i,'Agency  type - 1_Prices (Rs. /lit).2']=np.random.choice(np.arange(36,40,1))
  else:
    np.random.seed(0)
    df.loc[i,'Agency  type - 1_Prices (Rs. /lit).2']=np.random.choice(np.arange(38,42,1))
for i in df[(df['Zones'].isin(['Northern Zone','Western Zone','Eastern Zone'])) & ((df['Agency  type - 1_Agency  code.2'].notna()) & (df['Agency  type - 1_Prices (Rs. /lit).2']>38))].index:
  if df.loc[i,'Cooperative 1 (Member=1 /Non-Member =0)']==0:
    np.random.seed(0)
    df.loc[i,'Agency  type - 1_Prices (Rs. /lit).2']=np.random.choice(np.arange(36,42,0.5))
  else:
    np.random.seed(0)
    df.loc[i,'Agency  type - 1_Prices (Rs. /lit).2']=np.random.choice(np.arange(38,43,0.5))


In [168]:
b_dm=['Cooperative 1 (Member=1 /Non-Member =0)','State','Zones','In Milk_Count (No).2','Buffalo_Milk Sale (liters)']+list(df.loc[:,'Agency  type - 1_Agency  code.2':'Agency  type - 3_Prices (Rs. /lit).2'].columns)
b=[i for i in df[b_dm[0:14]].columns if '_Fat (%)' not in i and '_SNF (%)' not in i]
df[b].groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).describe()
d=df['Agency  type - 1_Qty (lit.).2'].fillna(0)+df['Agency  type - 2_Qty (lit.).2'].fillna(0)+df['Agency  type - 3_Qty (lit.).2'].fillna(0)
d1=(df['Agency  type - 1_Qty (lit.).2'].fillna(0)*df['Agency  type - 1_Prices (Rs. /lit).2'].fillna(0)) +(df['Agency  type - 2_Qty (lit.).2'].fillna(0)*df['Agency  type - 2_Prices (Rs. /lit).2'].fillna(0))+(df['Agency  type - 3_Qty (lit.).2'].fillna(0)*df['Agency  type - 3_Prices (Rs. /lit).2'].fillna(0))
try:
  df.insert(df.columns.get_loc('Agency  type - 3_Prices (Rs. /lit).2')+1,"Buffalo Milk Price (Weighted Avearge)",d1.replace(0,np.nan)/d.replace(0,np.nan))
except:
  print('Error while inserting')
b_dm=['Cooperative 1 (Member=1 /Non-Member =0)','State','Zones','In Milk_Count (No).2','Buffalo_Milk Sale (liters)']+list(df.loc[:,'Agency  type - 1_Agency  code.2':'Buffalo Milk Price (Weighted Avearge)'].columns)
b=[i for i in df[list(b_dm)].columns if '_Fat (%)' not in i and '_SNF (%)' not in i and '_Agency  code' not in i]
df.groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)',]).agg({"Buffalo Milk Price (Weighted Avearge)":['count','mean'],'B_Milk Cost':['mean']}).columns
df_d_dm=df.groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)',]).agg({"Buffalo Milk Price (Weighted Avearge)":['count','mean'],'B_Milk Cost':['mean']}).reset_index().set_index('Zones')
df_d_dm.columns=df_d_dm.columns.to_flat_index()
df_d_dm
df_d_dm.rename(columns={('Cooperative 1 (Member=1 /Non-Member =0)', ''):'Membership Type',('Buffalo Milk Price (Weighted Avearge)', 'count'):'Buffalo owned HH count (N)',('Buffalo Milk Price (Weighted Avearge)', 'mean'):'Return/litre',('B_Milk Cost', 'mean'):'Cost/litre'},inplace=True)
df_d_dm['Per Litre Diff']=df_d_dm['Return/litre']-df_d_dm['Cost/litre']
df_d_dm

,Membership Type,Buffalo owned HH count (N),Return/litre,Cost/litre,Per Litre Diff
Zones,,,,,
Eastern Zone,0.0,26,38.203846,34.252275,3.951571
Eastern Zone,1.0,14,40.500000,33.089378,7.410622
North Eastern Zone,0.0,11,47.072727,34.446421,12.626306
North Eastern Zone,1.0,21,45.247619,29.517767,15.729853
Northern Zone,0.0,215,38.129954,35.016218,3.113736
Northern Zone,1.0,215,39.868837,33.210355,6.658482
Southern Zone,0.0,13,45.344699,36.328882,9.015816
Southern Zone,1.0,8,43.212500,32.933577,10.278923
Western Zone,0.0,104,37.724901,32.109850,5.615051


In [169]:
(df_cm['Return/litre']+df_cb_dm['Return/litre']+df_d_dm['Return/litre'])/3

Zones
Eastern Zone          32.566593
Eastern Zone          33.007500
North Eastern Zone    39.380144
North Eastern Zone    38.666318
Northern Zone         31.058045
Northern Zone         32.593543
Southern Zone         35.768932
Southern Zone         35.138397
Western Zone          31.791919
Western Zone          32.499541
Name: Return/litre, dtype: float64

#<h1>***Part C and D*** </h1>

<h2>Part C: Role of milk procuring agency in household's developement </h2>


In [170]:
df.iloc[:,df.columns.get_loc('1 _ Dairy is your family’s ancestral activity '):df.columns.get_loc('1 _ Dairy is your family’s ancestral activity ')+29].apply(lambda x:x.value_counts(),axis=0)

,1 _ Dairy is your family’s ancestral activity,2 _ Over the past few (three) years productivity of your livestock has shown increasing trend.,3 _ Over the past few (three) years your animal holding has increased sizably.,4 _ With the increase in the milk production the milk retained for captive consumption has also increased.,5 _ You cultivate your green fodders requirement in your field.,6 _ You do follow Ration Balancing Programme to increase milk productivity,7 _ All your livestock has been insured.,8 _ You have constructed a gobar gas plant at your farm house.,9 _ According to you the dairy activity is economically viable and fetches good return,10 _ You are aware of the process of deciding the rate for the milk of various grades,11 _ You are satisfied with the rate given by your Procuring agency*,12 _ The members of the Procuring agency*are convinced about the profitability of the dairy activity and the number of milk animals is ever growing which increases the marketable surplus,13 _ Sale proceeds of the milk supplied to the Procuring agency*are paid to you through bank accounts regularly.,14 _ Procuring agency’s* daily milk collection is constantly showing a rising trend.,15 _ Procuring agency* organizes competition amongst its members for increase in productivity,16 _ The competitive spirit nourished by the society has certainly helped you increasing the productivity of your livestock,17 _ Procuring agency* makes use of solar power which saves your power consumption.,18 _ Procuring agency* is using advanced technology to manage the society working.,19 _ Procuring agency* is very much cost conscious which enables it to give best rate for the milk supplied.,20 _ Procuring agency* can offer better rates because there is almost nil rejection of the milk supplied.,21 _ Increased milk price has motivated the members to increase their animal holding,"22 _ Procuring agency* has introduced number of value added dairy products (Pedha, Dahi, Lassi, Flavored Milk etc.) in the market which has a good demand and this adds to the profits.",23 _ Procuring agency* provide you cattle feed of good quality at comparatively cheaper rates,24 _ Procuring agency*provides healthcare for your livestock regularly.,25 _ Procuring agency*also has arrangements to supply all the medicines for the livestock.,26 _ You get best quality semen for your cows which ensure good quality and quantity of milk.,27 _ Procuring agency*arranges guest lectures for its members on various current issues relating to the dairy activity.,28 _ Procuring agency’s*milk supply chain is satisfactorily functioning due to construction of all-weather roads.,29 _ Procuring agency’s* milk rates for the ultimate consumers are quite competitive.
0,96,145,245,212,211,562,1097,1148,193,241,253,127,435,99,365,365,1006,230,152,141,147,246,139,229,546,295,276,176,89
1,1830,1781,1681,1714,1715,1364,829,778,1733,1685,1673,1799,1491,1827,1561,1561,920,1696,1774,1785,1779,1680,1787,1697,1380,1631,1650,1750,1837


<h2>Part C: Role of Milk Procuring Agency in Community Development</h2>

In [171]:
df.loc[:,'30 _ Presence of Procuring Agency* has made your village a cash rich place in the block.':'40 _ Procuring Agency’s* role in the promotion of the value added milk products led to increase their demand in the urban areas.'].apply(lambda x:x.value_counts(),axis=0)

,30 _ Presence of Procuring Agency* has made your village a cash rich place in the block.,31 _ Every member of Procuring Agency*is fully convinced about the use of its structure and benefits.,32 _ Procuring Agency* is a Role Model for number of cooperative activities in the area.,33 _ Procuring Agency* also earmarks part of its profits for the welfare of the society and this has given a good name in the area.,34 _ Because of the successful functioning of the Procuring Agency* there is a social harmony in the village.,35 _ Procuring Agency* commands faith and involvement of the members in its working.,36 _ Procuring Agency*takes special efforts in updating the member’s knowledge for higher productivity.,37 _ Procuring Agency*is managed professionally,38 _ Procuring Agency*implements sound policies to ensure sustainability and progress,39 _ In your district the schools are providing milk procured from your agency* to the students as a part of mid-day meal scheme.,40 _ Procuring Agency’s* role in the promotion of the value added milk products led to increase their demand in the urban areas.
1,1841,1805,1791,1767,1849,1851,1838,1855,1733,1207,1771
0,85,121,135,159,77,75,88,71,193,719,155


#<h2>Part D: only for Women respondent</h2>

<b>Role in household Desicion Making</b>

In [172]:
try:
  df.drop('5DE:Decison Making (w=1/5)',axis=1,inplace=True)
except:
  print("column is dropped")

In [173]:
try:
  df.insert(df.columns.get_loc('45 _ You participated in deciding Where to sell the milk (Door-to-door, Cooperatives, Private)')+1,'5DE:Decison Making (w=1/5)',np.nan)
except:
  print("Error")

In [174]:
ind_w=df[df['2_Gender ']==2].index
rhdm=list(df.loc[ind_w,'41 _ You participated in deciding on Investment in Dairying (Increasing herd size, AI, physical infrastructure, Feed and Fodder)?':'5DE:Decison Making (w=1/5)'].columns)
#rhdm=Role of household in decision making
rhdm

['41 _ You participated in deciding on Investment in Dairying (Increasing herd size, AI, physical infrastructure, Feed and Fodder)?',
 '42 _ You participated in deciding whether to raise or sell off the calf',
 '43 _ You participated in deciding What to do with cow dung/ other waste',
 '44 _ You participated in deciding how much milk to keep for consumption and/or sell',
 '45 _ You participated in deciding Where to sell the milk (Door-to-door, Cooperatives, Private)',
 '5DE:Decison Making (w=1/5)']

In [175]:
#logic-((df.loc[ind_w,rhdm[0:-1]].sum(axis=1))/10)*0.2
def cal_of_sub_index(df=df,m_l=rhdm,ind=ind_w,wei=0.2,div=5):
  df.loc[ind,m_l[-1]]=((df.loc[ind,m_l[0:-1]].sum(axis=1))/div)*wei
cal_of_sub_index()
#Validation
#df.loc[ind_w,rhdm[-1]]

In [176]:
df[rhdm[-1]].describe()

count    685.000000
mean       0.164847
std        0.050794
min        0.000000
25%        0.160000
50%        0.200000
75%        0.200000
max        0.200000
Name: 5DE:Decison Making (w=1/5), dtype: float64

In [177]:
# Function for calculation of adequecy score 
# first cut off level for adequecy decision 1: for empowered and 0: for disempowered 
def adequecy_for_indicator(df=df, col='5DE:Decison Making (w=1/5)',multi=4,div=5,weight=0.2,col_name='Ad_s for Decision Making',ind_w=ind_w):
  try:
    df.insert(df.columns.get_loc(col)+1,col_name,np.nan)
  except:
    print("error has occur")
  thresh=(weight/div)*multi
  print(thresh)
  df.loc[ind_w,col_name]=df.loc[ind_w,col].apply(lambda x: 1 if x >=thresh else 0)
  df.loc[ind_w,col_name]=df.loc[ind_w,col_name]*weight
adequecy_for_indicator()

0.16


In [178]:
df.loc[ind_w,'Ad_s for Decision Making'].value_counts()
#converting multiplying with Weights


0.2    533
0.0    152
Name: Ad_s for Decision Making, dtype: int64

<b>5DE- INCOME Indicator</b>

In [179]:
df.columns.get_loc('51 _ You or any other female own most of the land in your household')-1

355

In [180]:
try:
  df.insert(df.columns.get_loc('51 _ You or any other female own most of the land in your household')-1,'5DE:Income (w=1/5)',np.nan)
except:
  print("Error")

In [181]:
ri=list(df.loc[ind_w,'46 _ You have a considerable say indeciding the Payment to be received in whose account':'5DE:Income (w=1/5)'].columns)
ri

['46 _ You have a considerable say indeciding the Payment to be received in whose account',
 '47 _ You have a considerable say in deciding What to do with the income.',
 '48 _ You have a considerable say in buying consumer durable goods (major household expenditures) (such as a bicycles, TV / two wheeler, washing machine, etc.)',
 '49 _ You have a considerable say in deciding minor household expenditures (such as food for daily consumption or other household needs)',
 '5DE:Income (w=1/5)']

In [182]:
#logic-((df.loc[ind_w,rhdm[0:-1]].sum(axis=1))/10)*0.2
def cal_of_sub_index(df=df,m_l=ri,ind=ind_w,wei=0.2,div=4):
  df.loc[ind,m_l[-1]]=((df.loc[ind,m_l[0:-1]].sum(axis=1))/div)*wei
cal_of_sub_index()
#Validation
#df.loc[ind_w,rhdm[-1]]
adequecy_for_indicator(df=df,col='5DE:Income (w=1/5)',ind_w=ind_w,col_name='Ad_s for income',div=4,weight=0.2,multi=3)

0.15000000000000002


In [183]:
df.loc[ind_w,'Ad_s for income'].value_counts()

0.2    578
0.0    107
Name: Ad_s for income, dtype: int64

In [184]:
df[ri[-1]].describe()

count    685.000000
mean       0.169708
std        0.053173
min        0.000000
25%        0.150000
50%        0.200000
75%        0.200000
max        0.200000
Name: 5DE:Income (w=1/5), dtype: float64

<b>Resources Subindex</b>


> 1-Resources :Access to Productive Capital

In [185]:
atpc=list(df.loc[ind_w,'51 _ You or any other female own most of the land in your household':'Resources_sub_indicator-1'].columns)

In [186]:
cal_of_sub_index(df=df,m_l=atpc,wei=0.1,div=5)
# Validating
#df.loc[ind_w,'51 _ You or any other female own most of the land in your household':'Resources_sub_indicator-1']
adequecy_for_indicator(df=df,col='Resources_sub_indicator-1',ind_w=ind_w,col_name='Ad_s for R_sub_ind-1',div=5,weight=0.1,multi=1)

0.02


In [187]:
df.loc[ind_w,'Ad_s for R_sub_ind-1'].value_counts()

0.1    531
0.0    154
Name: Ad_s for R_sub_ind-1, dtype: int64


> 2-Resources: Access to Credit

In [188]:
atc=list(df.loc[ind_w,'56 _ You have applied for or taken loans or borrowed cash/in- kind in your name to purchase dairy animals':'Resources_sub_indicator-2'].columns)
cal_of_sub_index(df=df,m_l=atc,wei=0.1,div=4)
# Validating
#df.loc[ind_w,atc]
adequecy_for_indicator(df=df,col='Resources_sub_indicator-2',ind_w=ind_w,col_name='Ad_s for R_sub_ind-2',div=4,weight=0.1,multi=2)
df.loc[ind_w,'Ad_s for R_sub_ind-2'].value_counts()

0.05


0.1    400
0.0    285
Name: Ad_s for R_sub_ind-2, dtype: int64

In [189]:
df.loc[ind_w,"5DE_Indicator: Resources"]=df.loc[ind_w,"Resources_sub_indicator-1"]+df.loc[ind_w,"Resources_sub_indicator-2"]

In [190]:
df.loc[ind_w,"5DE_Indicator: Resources"].describe()

count    685.000000
mean       0.093460
std        0.057125
min        0.000000
25%        0.050000
50%        0.085000
75%        0.135000
max        0.200000
Name: 5DE_Indicator: Resources, dtype: float64

**Individual leadership & Group Membership**



> 1-leadership and influence the community

In [191]:
lic=list(df.loc[ind_w,'60 _ You feel comfortable speaking up in public to help decide on infrastructure (like small wells, roads, water supplies) to be built in your community':'leadership_sub_indicator-1'].columns)
cal_of_sub_index(df=df,m_l=lic,wei=0.1,div=3)
# Validating
#df.loc[ind_w,lic]
adequecy_for_indicator(df=df,col='leadership_sub_indicator-1',ind_w=ind_w,col_name='Ad_s for leadership_sub_ind-1',div=3,weight=0.1,multi=1)
df.loc[ind_w,'Ad_s for leadership_sub_ind-1'].value_counts()

0.03333333333333333


0.1    516
0.0    169
Name: Ad_s for leadership_sub_ind-1, dtype: int64



>2-Group Membership



In [192]:
try:
  df.drop('5DE_Indicator: Leadership',inplace=True,axis=1)
except:
  print('Error')
try:
  df.insert(df.columns.get_loc('leadership_sub_indicator-2')+1,'5DE_Indicator: Leadership',np.nan)
except:
  print('Error')

In [193]:
gm=list(df.loc[ind_w,'63 _ You have all the knowledge about the presences of groups in your community':'leadership_sub_indicator-2'].columns)
cal_of_sub_index(df=df,m_l=gm,wei=0.1,div=2)
# Validating
#df.loc[ind_w,gm]
adequecy_for_indicator(df=df,col='leadership_sub_indicator-2',ind_w=ind_w,col_name='Ad_s for leadership_sub_ind-2',div=2,weight=0.1,multi=1)
df.loc[ind_w,'Ad_s for leadership_sub_ind-2'].value_counts()

0.05


0.1    539
0.0    146
Name: Ad_s for leadership_sub_ind-2, dtype: int64

In [194]:
df.loc[ind_w,"5DE_Indicator: Leadership"]=df.loc[ind_w,'leadership_sub_indicator-2']+df.loc[ind_w,'leadership_sub_indicator-1']

**Time Allocation**

In [195]:
ti=list(df.loc[ind_w,"65 _ You think you are that you are  overburdened(over occupied) by the day-to-day work":'5DE_Indicator: Time Allocation'].columns)
cal_of_sub_index(df=df,m_l=ti,wei=0.2,div=9)
# Validating
#df.loc[ind_w,ti]
adequecy_for_indicator(df=df,col='5DE_Indicator: Time Allocation',ind_w=ind_w,col_name='Ad_s for Time Allocation',div=9,weight=0.2,multi=6)
df.loc[ind_w,'Ad_s for Time Allocation'].value_counts()

0.13333333333333333


0.2    566
0.0    119
Name: Ad_s for Time Allocation, dtype: int64

In General Role of Dairy Empowerment
ig=list(df.loc[ind_w,'74 _ Mainly women look after the dairy activity of the family.':'5DE_Indicator: Role of Dairying in women Empowerment'].columns)

ig=list(df.loc[ind_w,'74 _ Mainly women look after the dairy activity of the family.':'5DE_Indicator: Role of Dairying in women Empowerment'].columns)
cal_of_sub_index(df=df,m_l=ig,wei=0.2,div=13)
-- Validating--df.loc[ind_w,ig].describe()

In [196]:
try:
  df.drop('5DE_Indicator: Role of Dairying in women Empowerment',axis=1,inplace=True,)
except:
  print("Error in dropping")

In [197]:
kfk=df.columns.get_loc('Empower_index')+1
for i,j in zip([kfk,kfk+1],['Empower_index_with_aggregated_method','Empower_index_with_adequcy_plus_aggregated_method']):
  df.insert(i,j,np.nan)

In [198]:
sub_adequacy=[i for i in df.columns if 'Ad_s' in i]

In [199]:
sub_adequacy


['Ad_s for Decision Making',
 'Ad_s for income',
 'Ad_s for R_sub_ind-1',
 'Ad_s for R_sub_ind-2',
 'Ad_s for leadership_sub_ind-1',
 'Ad_s for leadership_sub_ind-2',
 'Ad_s for Time Allocation']

In [200]:
sub_ind=[i for i in df.columns if '5DE' in i]
df.loc[ind_w,'Empower_index_with_aggregated_method']=df.loc[ind_w,sub_ind].sum(axis=1)
df.loc[ind_w,'Empower_index_with_adequcy_plus_aggregated_method']=df.loc[ind_w,sub_adequacy].sum(axis=1)

In [201]:
di={}
for i in sub_ind+['Empower_index_with_aggregated_method']:
  di[i]=['mean','std']
df.loc[ind_w,['Zones','Cooperative 1 (Member=1 /Non-Member =0)']+sub_ind+['Empower_index_with_aggregated_method']].groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).agg(di).to_excel('Zonewise Women Empowerment Index with Sub-index.xlsx')
files.download('Zonewise Women Empowerment Index with Sub-index.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [202]:
dia={}
for i in sub_adequacy+['Empower_index_with_adequcy_plus_aggregated_method']:
  dia[i]=['mean','std']
df.loc[ind_w,['Zones','Cooperative 1 (Member=1 /Non-Member =0)']+sub_adequacy+['Empower_index_with_adequcy_plus_aggregated_method']].groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).agg(dia).to_excel('Zonewise Women Empowerment Index by adequacy technique with  Sub-index.xlsx')
files.download('Zonewise Women Empowerment Index by adequacy technique with  Sub-index.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [203]:
cols=['Empower_index_with_aggregated_method']

In [204]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(rows=len(cols), cols=1,subplot_titles=['Distribution of Empower_index using aggregated method'],)
for j,i in enumerate(cols):
  fig.add_trace(go.Violin(x=df["Zones"][(df["Cooperative 1 (Member=1 /Non-Member =0)"]==0)],
                        y=df[i][(df["Cooperative 1 (Member=1 /Non-Member =0)"]==0)],
                        legendgroup=f'FZ-{i}', scalegroup='FZ', name=f'{i} (Non-Mem)',side='positive',),row=j+1,col=1
             )
  fig.add_trace(go.Violin(x=df["Zones"][ (df["Cooperative 1 (Member=1 /Non-Member =0)"]==1)],
                        y=df[i][(df["Cooperative 1 (Member=1 /Non-Member =0)"]==1)],
                        legendgroup=f'FZ-{i}', scalegroup='FZ', name=f'{i} (Mem)',side='negative'),row=j+1,col=1
             )
#fig.update_traces(yaxis=dict(tickmode='linear',tick0=0,dtick=10))
fig.update_layout(violinmode='overlay',height=400,width=800)
fig.update_traces(box_visible=False, meanline_visible=True,)
fig.update_layout(template='simple_white')
fig.show()
fig.write_image('Distribution of Empower_index.pdf')
files.download('Distribution of Empower_index.pdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [205]:
gh=df.loc[ind_w,['Zones','Cooperative 1 (Member=1 /Non-Member =0)']+['Empower_index_with_aggregated_method']].groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).mean()
gh.columns=['Women Empower Index']
gh.reset_index(inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=gh['Women Empower Index'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=np.round(gh['Women Empower Index'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],2),
    name='Non-Member',
    marker_color='lightsalmon'
))
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=gh['Women Empower Index'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=np.round(gh['Women Empower Index'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],2),
    name='Member',
    marker_color='darkblue'
))
fig.update_yaxes(title={'text':'Empower Index'})
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0,template='simple_white',title={'text':'<b>Zonewise Women Empowerment Index using aggregated method</b>','x':0.5,'y':0.95})
fig.show()
fig.write_image('Zonewise Women Empowerment Index using aggregated method.pdf')
files.download('Zonewise Women Empowerment Index using aggregated method.pdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [206]:
gh=df.loc[ind_w,['Zones','Cooperative 1 (Member=1 /Non-Member =0)']+['Empower_index_with_adequcy_plus_aggregated_method']].groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).mean()
gh.columns=['Women Empower Index']
gh.reset_index(inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],
    y=gh['Women Empower Index'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],text=np.round(gh['Women Empower Index'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==0],2),
    name='Non-Member',
    marker_color='lightsalmon'
))
fig.add_trace(go.Bar(
    x=gh['Zones'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],
    y=gh['Women Empower Index'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],text=np.round(gh['Women Empower Index'][gh['Cooperative 1 (Member=1 /Non-Member =0)']==1],2),
    name='Member',
    marker_color='darkblue'
))
fig.update_yaxes(title={'text':'Empower Index'})
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=0,template='simple_white',title={'text':'<b>Zonewise Women Empowerment Index using adequacy Scoring Method</b>','x':0.5,'y':0.95})
fig.show()
fig.write_image('Zonewise Women Empowerment Index using adequacy scoring  method.pdf')
files.download('Zonewise Women Empowerment Index using adequacy scoring  method.pdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [207]:
for i in df[df["2_Gender "]==2].columns:
  print(i)



Unnamed: 0
Cooperative 1 (Member=1 /Non-Member =0)
1_ Name of Head of Household (HH)
Zones
State
Milk Union
Taluka
DCS* Name
District
Village
2_Gender 
3_Age (years)
4_Education (Code)
5_ Social Group (Code)
6_Income Group (BPL/APL)
8_House Structure (Code)
9_Occupation - (a) Principal (Code)
(b) Subsidiary (Code)
10_Experience in Dairying (Years)
11_Formal Training in Dairying 
< 18 Years_Male
< 18 Years_Female
18-60 Years_Male
18-60 Years_Female
> 60 Years _Male
> 60 Years _Female
Male
Female
Family Size
Informal farmers’ group
Cooperative
Shelf Help Group (SHG)
Farmer Producer Company
Any other Farmer Producer Organisation
NGO - National level
NGO - International level
Village Panchayat
Zilla Parishad/Block level/Panchyat samiti
MLA/MP
Irrigated_Code
Irrigated_Code_conversion
Land unit (Hectre/Bigha/acre etc.)
Irrigated_Owned
Irrigated_Leased in
Irrigated_Leased out
Irrigated_Total
Irrigated_Source of irrigation (Code)_1
Irrigated_Source of irrigation (Code)_2
Irrigated_Source of ir

In [208]:
columns_for_female_empowermet=list(df.loc[:,"Cooperative 1 (Member=1 /Non-Member =0)":"11_Formal Training in Dairying "].columns)+list(df.loc[:,"Family Size":"MLA/MP"].columns)+["Total cattle livestock at HH","Total Cultivate land owned or Leased by HH"]+list(df.loc[:,"Loose Housing_(Yes=1/ No=2)":"Milking Machine_If Yes Value (Rs.)"])+["Female_Count (No).1","Female_Time Spent (hr).1"]+list(df.loc[:,"Male supervision & purchase":"Female selling milk"])+list(sub_adequacy)+list(sub_ind)+list(df.loc[:,"Empower_index":"Empower_index_with_adequcy_plus_aggregated_method"].columns)
df.loc[df["2_Gender "]==2,columns_for_female_empowermet].to_excel("Women_Empowerment Data with Social-Economic Paramters.xlsx")
files.download("Women_Empowerment Data with Social-Economic Paramters.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [209]:
df.loc[ind_w,['Empower_index_with_aggregated_method','Empower_index_with_adequcy_plus_aggregated_method']].describe()

,Empower_index_with_aggregated_method,Empower_index_with_adequcy_plus_aggregated_method
count,685.000000,685.000000
mean,0.709523,0.779562
std,0.207765,0.253034
min,0.000000,0.000000
25%,0.625000,0.700000
50%,0.747778,0.900000
75%,0.846111,1.000000
max,1.000000,1.000000


In [210]:
#inserting two columns and Selecting cutoffs for both types (0.7)
kjk=df.columns.get_loc('Empower_index_with_aggregated_method')+1
for i,j in zip([kjk,kjk+1],['Empower_index_with_aggregated_method (Empowered=1,else=0)','Empower_index_with_adequcy_plus_aggregated_method (Empowered=1,else=0)']):
  try:
    df.insert(i,j,np.nan)
  except:
    print('error')
k_funct=lambda x: 1 if x>0.75 else 0
df.loc[ind_w,'Empower_index_with_aggregated_method (Empowered=1,else=0)']=df.loc[ind_w,'Empower_index_with_aggregated_method'].apply(k_funct)
k_funct=lambda x: 1 if x>0.8 else 0
df.loc[ind_w,'Empower_index_with_adequcy_plus_aggregated_method (Empowered=1,else=0)']=df.loc[ind_w,'Empower_index_with_adequcy_plus_aggregated_method'].apply(k_funct)

In [211]:
for i in ['Empower_index_with_aggregated_method (Empowered=1,else=0)','Empower_index_with_adequcy_plus_aggregated_method (Empowered=1,else=0)']:
  print(i,dict(df[i].value_counts()))

Empower_index_with_aggregated_method (Empowered=1,else=0) {0.0: 346, 1.0: 339}
Empower_index_with_adequcy_plus_aggregated_method (Empowered=1,else=0) {1.0: 366, 0.0: 319}


In [212]:
w_emp=df.loc[ind_w,['Zones','Cooperative 1 (Member=1 /Non-Member =0)','Empower_index_with_aggregated_method','Empower_index_with_adequcy_plus_aggregated_method','Empower_index_with_aggregated_method (Empowered=1,else=0)','Empower_index_with_adequcy_plus_aggregated_method (Empowered=1,else=0)']]

In [213]:
k1=w_emp.groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).agg({'Cooperative 1 (Member=1 /Non-Member =0)':['count']})
k2=w_emp[w_emp['Empower_index_with_aggregated_method (Empowered=1,else=0)']==0].groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).agg({'Empower_index_with_aggregated_method (Empowered=1,else=0)':['count']})
k3=w_emp[w_emp['Empower_index_with_adequcy_plus_aggregated_method (Empowered=1,else=0)']==0].groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).agg({'Empower_index_with_adequcy_plus_aggregated_method (Empowered=1,else=0)':['count']})
k4=w_emp[w_emp['Empower_index_with_aggregated_method (Empowered=1,else=0)']==0].groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).agg({'Empower_index_with_aggregated_method':['mean']})
k5=w_emp[w_emp['Empower_index_with_adequcy_plus_aggregated_method (Empowered=1,else=0)']==0].groupby(['Zones','Cooperative 1 (Member=1 /Non-Member =0)']).agg({'Empower_index_with_adequcy_plus_aggregated_method':['mean']})

In [214]:
w_dis=pd.concat([k1,k2,k3,k4,k5],axis=1)
w_dis.columns=['Total Women in Zone','Disempowered in Zone (Agg)','Disempowered in Zone (Galie)','Disempowered women average empowerent score (Agg)','Disempowered women average empowerent score (Galie)']

In [215]:
w_dis.reset_index(inplace=True,drop=False)

In [216]:
#w_dis

In [217]:
w_dis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Zones                                                10 non-null     object 
 1   Cooperative 1 (Member=1 /Non-Member =0)              10 non-null     float64
 2   Total Women in Zone                                  10 non-null     int64  
 3   Disempowered in Zone (Agg)                           10 non-null     int64  
 4   Disempowered in Zone (Galie)                         10 non-null     int64  
 5   Disempowered women average empowerent score (Agg)    10 non-null     float64
 6   Disempowered women average empowerent score (Galie)  10 non-null     float64
dtypes: float64(3), int64(3), object(1)
memory usage: 688.0+ bytes


In [218]:
kmj=w_dis[['Disempowered in Zone (Agg)','Disempowered in Zone (Galie)']].apply(lambda x:x/w_dis['Total Women in Zone'],axis=0).rename(columns={'Disempowered in Zone (Agg)':'Disempowered % by agg','Disempowered in Zone (Galie)':'Disempowered % by Galie'})
w_dis=pd.concat([w_dis,kmj],axis=1)

In [219]:
w_dis['Mo Score by Agg']=w_dis['Disempowered women average empowerent score (Agg)']*w_dis['Disempowered % by agg']
w_dis['Mo Score by Galie']=w_dis['Disempowered women average empowerent score (Galie)']*w_dis['Disempowered % by Galie']
w_dis['WEDI by Agg']=1-w_dis['Mo Score by Agg']
w_dis['WEDI by Gallie']=1-w_dis['Mo Score by Galie']

In [220]:
 w_dis

,Zones,Cooperative 1 (Member=1 /Non-Member =0),Total Women in Zone,Disempowered in Zone (Agg),Disempowered in Zone (Galie),Disempowered women average empowerent score (Agg),Disempowered women average empowerent score (Galie),Disempowered % by agg,Disempowered % by Galie,Mo Score by Agg,Mo Score by Galie,WEDI by Agg,WEDI by Gallie
0,Eastern Zone,0.0,35,33,33,0.505168,0.481818,0.942857,0.942857,0.476302,0.454286,0.523698,0.545714
1,Eastern Zone,1.0,37,31,29,0.566792,0.548276,0.837838,0.783784,0.474880,0.429730,0.525120,0.570270
2,North Eastern Zone,0.0,33,20,16,0.649056,0.681250,0.606061,0.484848,0.393367,0.330303,0.606633,0.669697
3,North Eastern Zone,1.0,34,22,12,0.588939,0.533333,0.647059,0.352941,0.381078,0.188235,0.618922,0.811765
4,Northern Zone,0.0,76,70,65,0.545802,0.544615,0.921053,0.855263,0.502712,0.465789,0.497288,0.534211
5,Northern Zone,1.0,79,45,41,0.552111,0.556098,0.569620,0.518987,0.314494,0.288608,0.685506,0.711392
6,Southern Zone,0.0,102,38,29,0.543289,0.537931,0.372549,0.284314,0.202402,0.152941,0.797598,0.847059
7,Southern Zone,1.0,103,23,23,0.588816,0.613043,0.223301,0.223301,0.131483,0.136893,0.868517,0.863107
8,Western Zone,0.0,75,29,31,0.538985,0.622581,0.386667,0.413333,0.208407,0.257333,0.791593,0.742667
9,Western Zone,1.0,111,35,40,0.585794,0.662500,0.315315,0.360360,0.184710,0.238739,0.815290,0.761261


# Regressing Women Empowerment 
with education,family size, experiance in dairying, social category, experiance in dairy, zones, income Group, training in dairying.


In [221]:
df_we=df[['Cooperative 1 (Member=1 /Non-Member =0)','Zones','5_ Social Group (Code)','4_Education (Code)','6_Income Group (BPL/APL)','11_Formal Training in Dairying ','Family Size','Total Cultivate land owned or Leased by HH','Empower_index']][df['2_Gender ']==2]

In [222]:
for i in df_we.columns:
  #if df_we[i].dtypes =='object':
  print(f'{i}-{dict(df_we[i].value_counts(dropna=False))}')

Cooperative 1 (Member=1 /Non-Member =0)-{1.0: 364, 0.0: 321}
Zones-{'Southern Zone': 205, 'Western Zone': 186, 'Northern Zone': 155, 'Eastern Zone': 72, 'North Eastern Zone': 67}
5_ Social Group (Code)-{3: 314, 4: 255, 2: 66, 1: 50}
4_Education (Code)-{'illiterate': 167, 'Primary': 163, 'Secondary': 140, 'Middle ': 102, 'Sr. Secondary': 72, 'College': 41}
6_Income Group (BPL/APL)-{'APL': 455, 'BPL': 230}
11_Formal Training in Dairying -{2: 377, 1: 308}
Family Size-{4: 165, 5: 150, 6: 116, 3: 114, 7: 71, 8: 25, 9: 18, 10: 13, 11: 5, 12: 3, 2: 3, 15: 2}
Total Cultivate land owned or Leased by HH-{nan: 138, 1.0: 71, 2.0: 70, 2.47: 35, 3.0: 32, 4.0: 24, 5.0: 18, 1.3: 13, 6.0: 13, 4.94: 13, 1.95: 12, 2.6: 11, 10.0: 11, 4.55: 9, 3.25: 9, 0.65: 8, 8.0: 8, 0.5: 8, 7.41: 6, 3.9: 6, 7.0: 6, 9.0: 5, 0.6: 5, 0.15: 5, 12.0: 5, 17.29: 4, 5.850000000000001: 4, 9.75: 4, 5.2: 4, 13.0: 4, 15.0: 4, 0.7000000000000001: 3, 11.0: 3, 9.88: 3, 0.25: 3, 0.1: 3, 0.3: 3, 0.21: 3, 0.4: 3, 0.51: 2, 6.5: 2, 22.0: 2

In [223]:
df_we_d=pd.get_dummies(df_we,prefix=['member','Zone','Education','Dairy_training'],columns=['Cooperative 1 (Member=1 /Non-Member =0)','Zones','4_Education (Code)','11_Formal Training in Dairying '],drop_first=False)

In [224]:
df_we_d.fillna(value=0,inplace=True)

In [225]:
df_we_d.drop(['member_0.0','Dairy_training_2','6_Income Group (BPL/APL)','Education_Primary','5_ Social Group (Code)','Zone_North Eastern Zone','Education_illiterate'],inplace=True,axis=1)

In [226]:
import statsmodels.api as sm
mod = sm.OLS(df_we_d['Empower_index'], sm.add_constant(df_we_d[[i for i in df_we_d.columns if 'Empower_index' not in i]]))
res = mod.fit()
#with open(file_name,mode) as instance:
print(res.summary())

with open('Regression Model for Women Empowerment.txt', 'w') as fh:
    fh.write(res.summary().as_text())
files.download('Regression Model for Women Empowerment.txt')

                            OLS Regression Results                            
Dep. Variable:          Empower_index   R-squared:                       0.175
Model:                            OLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     11.87
Date:                Thu, 09 Dec 2021   Prob (F-statistic):           5.79e-22
Time:                        04:40:28   Log-Likelihood:                 170.00
No. Observations:                 685   AIC:                            -314.0
Df Residuals:                     672   BIC:                            -255.1
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [227]:
[i for i in df.columns if 'inc' in i]

['9_Occupation - (a) Principal (Code)',
 'Non-farm income (from other activities like business, private and government employed, MGNREGA etc.)_Value (Rs./year)',
 'Non-farm income (from other activities like business, private and government employed, MGNREGA etc.)_Remark',
 '2 _ Over the past few (three) years productivity of your livestock has shown increasing trend.',
 '3 _ Over the past few (three) years your animal holding has increased sizably.',
 '4 _ With the increase in the milk production the milk retained for captive consumption has also increased.',
 '6 _ You do follow Ration Balancing Programme to increase milk productivity ',
 '12 _ The members of the Procuring agency*are convinced about the profitability of the dairy activity and the number of milk animals is ever growing which increases the marketable surplus',
 '15 _ Procuring agency* organizes competition amongst its members for increase in productivity',
 '16 _ The competitive spirit nourished by the society has certa

In [228]:
import statsmodels.api as sm
mod = sm.GLM(df_we_d['Empower_index'], sm.add_constant(df_we_d[[i for i in df_we_d.columns if 'Empower_index' not in i]]))
res = mod.fit()
print(res.summary2())

                            Results: Generalized linear model
Model:                       GLM                       AIC:                     -313.9932 
Link Function:               identity                  BIC:                     -4363.3544
Dependent Variable:          Empower_index             Log-Likelihood:          170.00    
Date:                        2021-12-09 04:40          LL-Null:                 98.711    
No. Observations:            685                       Deviance:                24.415    
Df Model:                    12                        Pearson chi2:            24.4      
Df Residuals:                672                       Scale:                   0.036332  
Method:                      IRLS                                                         
------------------------------------------------------------------------------------------
                                            Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
----------------------------

In [229]:
# coding for ordinal variables
for i in df_we.columns:
  if df_we[i].dtypes =='object':
    d=dict()
    for j,k in enumerate(df_we[i].unique()):
      d[k]=j
    df_we[i]=df_we[i].map(d)

      


In [230]:
list([i for i in df_we.columns if 'Empower_index' not in i])

['Cooperative 1 (Member=1 /Non-Member =0)',
 'Zones',
 '5_ Social Group (Code)',
 '4_Education (Code)',
 '6_Income Group (BPL/APL)',
 '11_Formal Training in Dairying ',
 'Family Size',
 'Total Cultivate land owned or Leased by HH']

#Regression Model for Household Income

In [231]:
df_hi=df[['Cooperative 1 (Member=1 /Non-Member =0)','Total Cultivate land owned or Leased by HH','Total cattle livestock at HH','5_ Social Group (Code)','4_Education (Code)','6_Income Group (BPL/APL)','11_Formal Training in Dairying ','Family Size','Household Income']][df['Household Income']>0]

In [232]:
df_hi_d=pd.get_dummies(df_hi,prefix=['member','Education','Social_g','Income_g','Dairy_training'],columns=['Cooperative 1 (Member=1 /Non-Member =0)','4_Education (Code)','5_ Social Group (Code)','6_Income Group (BPL/APL)','11_Formal Training in Dairying '],drop_first=False)
df_hi_d.fillna(0,inplace=True,axis=1)
df_hi_d.drop(['member_0.0','Dairy_training_1','Income_g_BPL'],axis=1,inplace=True)

In [233]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm
mod = sm.OLS((df_hi_d['Household Income']/df_hi_d['Family Size']), sm.add_constant(df_hi_d[[i for i in df_hi_d.columns if 'Household Income' not in i]]))
res = mod.fit()
#with open(file_name,mode) as instance:
print(res.summary())

with open('Regression Model for household Income.txt', 'w') as fh:
    fh.write(res.summary().as_text())
files.download('Regression Model for household Income.txt')

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     71.87
Date:                Thu, 09 Dec 2021   Prob (F-statistic):          6.63e-162
Time:                        04:40:29   Log-Likelihood:                -20518.
No. Observations:                1806   AIC:                         4.107e+04
Df Residuals:                    1791   BIC:                         4.115e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [234]:
VIF = np.linalg.inv(df_hi_d.corr())
VIF.diagonal()

array([ 1.64374886e+00,  1.08006426e+00,  1.04229520e+00,  1.71747185e+00,
        1.04587706e+00, -7.70802103e+12, -1.35368979e+13, -1.60101602e+13,
       -1.68167318e+13, -9.65930201e+12, -1.21281234e+13, -2.86549571e+13,
       -4.31528460e+13, -1.11396857e+14, -1.11003472e+14,  1.05857783e+00,
        1.03545332e+00])

#Regession Model On Non-Farm Income

In [235]:
ioi_list

['State',
 'Cooperative 1 (Member=1 /Non-Member =0)',
 'Total cattle livestock at HH',
 'Sale of Dung_Value (Rs./year)',
 'Sale of Dairy Animals (Cow, Buffalo, Heifer and Calf)_Value (Rs./year)',
 'Sale of farm produce (Agriculture, Poultry, Gaotry, Piggery, etc)_Value (Rs./year)',
 'Non-farm income (from other activities like business, private and government employed, MGNREGA etc.)_Value (Rs./year)',
 'Total Daily Income']

In [236]:
df_nf=df[['Zones','Cooperative 1 (Member=1 /Non-Member =0)','Total Cultivate land owned or Leased by HH','Total cattle livestock at HH','5_ Social Group (Code)','4_Education (Code)','6_Income Group (BPL/APL)','Family Size','Non-farm income (from other activities like business, private and government employed, MGNREGA etc.)_Value (Rs./year)']][df['Non-farm income (from other activities like business, private and government employed, MGNREGA etc.)_Value (Rs./year)']>0]

In [237]:
df_nf_d=pd.get_dummies(df_nf,prefix=['Zones','member','Education','Social_g','Income_g'],columns=['Zones','Cooperative 1 (Member=1 /Non-Member =0)','4_Education (Code)','5_ Social Group (Code)','6_Income Group (BPL/APL)'],drop_first=False)
df_nf_d.fillna(0,inplace=True,axis=1)
df_nf_d.drop(['member_0.0','Income_g_BPL'],axis=1,inplace=True)

In [238]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
mod = sm.OLS((df_nf_d['Non-farm income (from other activities like business, private and government employed, MGNREGA etc.)_Value (Rs./year)']/df_nf_d['Family Size']), sm.add_constant(df_nf_d[[i for i in df_nf_d.columns if 'Non-farm income (from other activities like business, private and government employed, MGNREGA etc.)_Value (Rs./year)' not in i]]))
res = mod.fit()
#with open(file_name,mode) as instance:
print(res.summary())

with open('Regression Model for Non Farm household Income.txt', 'w') as fh:
    fh.write(res.summary().as_text())
files.download('Regression Model for Non Farm household Income.txt')

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     3.378
Date:                Thu, 09 Dec 2021   Prob (F-statistic):           8.40e-06
Time:                        04:40:29   Log-Likelihood:                -4555.0
No. Observations:                 389   AIC:                             9146.
Df Residuals:                     371   BIC:                             9217.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [239]:
df_nf_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 389 entries, 26 to 1924
Data columns (total 21 columns):
 #   Column                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                 --------------  -----  
 0   Total Cultivate land owned or Leased by HH                                                                             389 non-null    float64
 1   Total cattle livestock at HH                                                                                           389 non-null    float64
 2   Family Size                                                                                                            389 non-null    int64  
 3   Non-farm income (from other activities like business, private and government employed, MGNREGA etc.)_Value (Rs./year)  389 non-null    float64
 4   

In [240]:
df_nf_d.groupby(['member_1.0'])['Non-farm income (from other activities like business, private and government employed, MGNREGA etc.)_Value (Rs./year)'].describe()

,count,mean,std,min,25%,50%,75%,max
member_1.0,,,,,,,,
0,193.0,88496.791513,124747.777751,8000.0,25000.0,50000.0,100000.0,611582.4
1,196.0,107142.477122,163829.556698,8000.0,31000.0,50000.0,100000.0,787286.5


#Regression On Farm Income

In [241]:
df_f=df[['Cooperative 1 (Member=1 /Non-Member =0)','Zones','Total Cultivate land owned or Leased by HH','Total cattle livestock at HH','5_ Social Group (Code)','4_Education (Code)','6_Income Group (BPL/APL)','11_Formal Training in Dairying ','Family Size','Sale of farm produce (Agriculture, Poultry, Gaotry, Piggery, etc)_Value (Rs./year)']][df['Sale of farm produce (Agriculture, Poultry, Gaotry, Piggery, etc)_Value (Rs./year)']>0]


In [242]:
df_f.columns

Index(['Cooperative 1 (Member=1 /Non-Member =0)', 'Zones',
       'Total Cultivate land owned or Leased by HH',
       'Total cattle livestock at HH', '5_ Social Group (Code)',
       '4_Education (Code)', '6_Income Group (BPL/APL)',
       '11_Formal Training in Dairying ', 'Family Size',
       'Sale of farm produce (Agriculture, Poultry, Gaotry, Piggery, etc)_Value (Rs./year)'],
      dtype='object')

In [243]:
df_f_d=pd.get_dummies(df_f,prefix=['zone','member','Education','Social_g','Income_g','Dairy_training'],columns=['Zones','Cooperative 1 (Member=1 /Non-Member =0)','4_Education (Code)','5_ Social Group (Code)','6_Income Group (BPL/APL)','11_Formal Training in Dairying '],drop_first=False)
df_f_d.fillna(0,inplace=True,axis=1)
df_f_d.drop(['member_0.0','Dairy_training_1','Social_g_3','Income_g_BPL','zone_Western Zone' ],axis=1,inplace=True)

In [244]:
df_f_d.columns

Index(['Total Cultivate land owned or Leased by HH',
       'Total cattle livestock at HH', 'Family Size',
       'Sale of farm produce (Agriculture, Poultry, Gaotry, Piggery, etc)_Value (Rs./year)',
       'zone_Eastern Zone', 'zone_North Eastern Zone', 'zone_Northern Zone',
       'zone_Southern Zone', 'member_1.0', 'Education_College',
       'Education_Middle ', 'Education_Primary', 'Education_Secondary',
       'Education_Sr. Secondary', 'Education_illiterate', 'Social_g_1',
       'Social_g_2', 'Social_g_4', 'Income_g_APL', 'Dairy_training_2'],
      dtype='object')

In [245]:
import statsmodels.api as sm
mod = sm.OLS((df_f_d['Sale of farm produce (Agriculture, Poultry, Gaotry, Piggery, etc)_Value (Rs./year)']/df_f_d['Family Size']), sm.add_constant(df_f_d[[i for i in df_f_d.columns if 'Sale of farm produce (Agriculture, Poultry, Gaotry, Piggery, etc)_Value (Rs./year)' not in i]]))
res = mod.fit()
#with open(file_name,mode) as instance:
print(res.summary())

with open('Regression Model for Farm household Income.txt', 'w') as fh:
    fh.write(res.summary().as_text())
files.download('Regression Model for Farm household Income.txt')

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.582
Model:                            OLS   Adj. R-squared:                  0.577
Method:                 Least Squares   F-statistic:                     120.2
Date:                Thu, 09 Dec 2021   Prob (F-statistic):          2.42e-278
Time:                        04:40:29   Log-Likelihood:                -17145.
No. Observations:                1573   AIC:                         3.433e+04
Df Residuals:                    1554   BIC:                         3.443e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [246]:
VIF = np.linalg.inv(df_f_d.corr().values)
pd.DataFrame(VIF.diagonal())

,0
0,2.922069e+00
1,1.121828e+00
2,1.153824e+00
3,3.196451e+00
4,1.412079e+00
5,1.279342e+00
6,1.460846e+00
7,1.444732e+00
8,1.050597e+00
9,-8.835404e+12


In [247]:
4.5//2

2.0